<a href="https://colab.research.google.com/github/lambdabypi/MiniQuest/blob/main/MiniQuest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ==========================================
# MINIQUEST: AI-POWERED ADVENTURE GENERATOR
# ==========================================

In [ ]:
# Cell 1

# Installation & Setup
!pip install -q openai anthropic chromadb gradio python-dotenv requests python-dateutil googlemaps pytz langgraph langchain langchain-openai langchain-anthropic

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Cell 2

# Imports
import os
import json
import requests
from typing import List, Dict, Optional, TypedDict, Annotated
from datetime import datetime, timedelta
from dataclasses import dataclass, field
import gradio as gr
from enum import Enum
import pytz
import operator
import uuid
import hashlib
from collections import Counter, defaultdict

# LLM APIs
import openai
from anthropic import Anthropic

# LangGraph & LangChain
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage

# Vector Store
import chromadb
from chromadb.utils import embedding_functions

print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


In [ ]:
# Cell 3

# API Configuration
from google.colab import userdata

# Set up API keys
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
GOOGLE_MAPS_KEY = userdata.get('GOOGLE_MAPS_KEY')

# Configure APIs
openai.api_key = OPENAI_API_KEY
anthropic_client = Anthropic(api_key=ANTHROPIC_API_KEY)

print("✅ API keys configured (OpenAI + Claude)")

✅ API keys configured (OpenAI + Claude)


In [ ]:

# Cell 4

# Data Models
@dataclass
class AdventureParams:
    """Structured parameters extracted from user input"""
    mood: str
    time_available: int
    budget: float
    location: str
    energy_level: str
    preferences: List[str]
    constraints: List[str]
    weather_preference: str

@dataclass
class POI:
    """Point of Interest"""
    id: str
    name: str
    type: str
    description: str
    lat: float
    lon: float
    rating: float
    price_level: int
    current_wait: int
    tags: List[str]
    match_score: int = 0
    score_reason: str = ""
    distance_from_user: Optional[float] = None
    reviews: List[Dict] = field(default_factory=list)

@dataclass
class AdventureOption:
    """A complete adventure itinerary"""
    title: str
    tagline: str
    description: str
    match_score: int
    duration: int
    cost: float
    distance: float
    steps: List[Dict]
    reasoning: str
    weather_dependent: bool
    review_theme: str = ""
    review_example: str = ""
    review_rating: float = 0.0
    review_author: str = ""

@dataclass
class EnhancedPOI:
    """POI with image support"""
    id: str
    name: str
    type: str
    description: str
    lat: float
    lon: float
    rating: float
    price_level: int
    current_wait: int
    tags: List[str]
    match_score: int = 0
    score_reason: str = ""
    distance_from_user: Optional[float] = None
    images: List[Dict] = field(default_factory=list)
    photo_reference: str = ""
    reviews: List[Dict] = field(default_factory=list)

# LangGraph State Definition
class AdventureState(TypedDict):
    """Shared state across all agents in the workflow"""
    user_input: str
    address_input: Optional[str]
    lat: Optional[float]
    lon: Optional[float]
    params: Optional[AdventureParams]
    pois: List[POI]
    adventures: List[AdventureOption]
    context: Dict
    maps_html: List[str]
    metadata: Dict
    error: Optional[str]
    pipeline_log: List[str]

print("✅ Data models and LangGraph state defined")

✅ Data models and LangGraph state defined


In [ ]:
# Code 4.5 - ChromaDB RAG System

class MiniQuestRAGSystem:
    """Enhanced RAG system with robust user tracking and learning capabilities"""

    def __init__(self):
        """Initialize ChromaDB collections with enhanced user tracking"""

        # Initialize ChromaDB client (persistent storage)
        self.client = chromadb.Client()

        # OpenAI embedding function
        self.embedding_function = embedding_functions.OpenAIEmbeddingFunction(
            api_key=OPENAI_API_KEY,
            model_name="text-embedding-3-small"
        )

        # Collection 1: Boston Insider Tips (unchanged)
        try:
            self.client.delete_collection("boston_insider_tips")
        except:
            pass

        self.tips_collection = self.client.create_collection(
            name="boston_insider_tips",
            embedding_function=self.embedding_function,
            metadata={"description": "Curated Boston local knowledge and insider tips"}
        )

        # Collection 2: Enhanced User Adventure History
        try:
            self.client.delete_collection("user_adventure_history")
        except:
            pass

        self.history_collection = self.client.create_collection(
            name="user_adventure_history",
            embedding_function=self.embedding_function,
            metadata={"description": "User's past adventures with detailed preferences"}
        )

        # Collection 3: NEW - User Preference Profiles
        try:
            self.client.delete_collection("user_preference_profiles")
        except:
            pass

        self.profile_collection = self.client.create_collection(
            name="user_preference_profiles",
            embedding_function=self.embedding_function,
            metadata={"description": "Aggregated user preference profiles"}
        )

        # Collection 4: NEW - Location Context Memory
        try:
            self.client.delete_collection("location_context_memory")
        except:
            pass

        self.location_collection = self.client.create_collection(
            name="location_context_memory",
            embedding_function=self.embedding_function,
            metadata={"description": "User's location patterns and area preferences"}
        )

        # Session tracking
        self.current_session_data = {}

        # Seed with Boston insider knowledge
        self._seed_boston_tips()

        print("✅ Enhanced RAG system initialized with 4 collections")

    def _normalize_user_id(self, user_id: str) -> str:
        """Normalize user ID to handle variations and prevent duplicates"""
        if not user_id or not user_id.strip():
            return "anonymous_user"

        # Normalize: lowercase, strip whitespace, replace spaces with underscores
        normalized = user_id.strip().lower().replace(' ', '_').replace('.', '_')

        # Remove any non-alphanumeric characters except underscores
        import re
        normalized = re.sub(r'[^a-zA-Z0-9_]', '', normalized)

        return normalized or "anonymous_user"

    def _generate_adventure_id(self, user_id: str, adventure_title: str) -> str:
        """Generate unique adventure ID"""
        timestamp = datetime.now(pytz.timezone('America/New_York')).strftime('%Y%m%d_%H%M%S')
        content_hash = hashlib.md5(f"{user_id}_{adventure_title}".encode()).hexdigest()[:8]
        return f"{user_id}_adventure_{timestamp}_{content_hash}"

    def _seed_boston_tips(self):
        """Seed database with Boston insider knowledge (unchanged from your code)"""

        boston_tips = [
            {
                "id": "tip_001",
                "category": "coffee",
                "tip": "Tatte Bakery is a local favorite for European-style pastries and coffee. The Back Bay location has the best ambiance with large windows.",
                "metadata": {"vibe": "cozy", "price": "moderate", "neighborhood": "Back Bay"}
            },
            {
                "id": "tip_002",
                "category": "coffee",
                "tip": "George Howell Coffee in Newbury Street is where coffee nerds go. They roast their own beans and baristas are serious about their craft.",
                "metadata": {"vibe": "sophisticated", "price": "high", "neighborhood": "Back Bay"}
            },
            # ... (rest of your tips remain the same)
            {
                "id": "tip_020",
                "category": "budget_tips",
                "tip": "Boston Public Library (Copley Square) is stunning and free. The courtyard is a hidden oasis. Great for working or reading.",
                "metadata": {"vibe": "peaceful", "price": "free", "neighborhood": "Back Bay"}
            }
        ]

        # Add tips to collection
        for tip in boston_tips:
            self.tips_collection.add(
                ids=[tip["id"]],
                documents=[tip["tip"]],
                metadatas=[{
                    "category": tip["category"],
                    **tip["metadata"]
                }]
            )

        print(f"✅ Seeded {len(boston_tips)} Boston insider tips")

    def store_user_adventure(self, user_id: str, adventure: 'AdventureOption',
                            params: 'AdventureParams', rating: float = None,
                            completion_status: str = "completed", notes: str = ""):
        """ENHANCED: Store completed adventure with better tracking"""

        try:
            normalized_user_id = self._normalize_user_id(user_id)
            adventure_id = self._generate_adventure_id(normalized_user_id, adventure.title)

            # Create rich document for embedding with more context
            adventure_doc = f"""
            Adventure: {adventure.title}
            User: {normalized_user_id}
            Preferences: {', '.join(params.preferences)}
            Mood: {params.mood}
            Energy Level: {params.energy_level}
            Weather Preference: {params.weather_preference}
            Budget: ${params.budget}
            Duration: {params.time_available} minutes
            Actual Cost: ${adventure.cost}
            Actual Duration: {adventure.duration} minutes
            Theme: {adventure.reasoning}
            Match Score: {adventure.match_score}%
            Locations visited: {', '.join([step['activity'].split(':')[0] for step in adventure.steps])}
            Steps: {'; '.join([f"{step['time']} - {step['activity']}" for step in adventure.steps])}
            """

            if rating:
                adventure_doc += f"\nUser Rating: {rating}/5 stars"
            if notes:
                adventure_doc += f"\nUser Notes: {notes}"
            if completion_status != "completed":
                adventure_doc += f"\nStatus: {completion_status}"

            # Enhanced metadata with more tracking
            boston_tz = pytz.timezone('America/New_York')
            current_time = datetime.now(boston_tz)

            metadata = {
                "user_id": normalized_user_id,
                "adventure_title": adventure.title,
                "preferences": ','.join(params.preferences),
                "mood": params.mood,
                "energy_level": params.energy_level,
                "budget": float(params.budget),
                "time_requested": int(params.time_available),
                "actual_cost": float(adventure.cost),
                "actual_duration": int(adventure.duration),
                "match_score": int(adventure.match_score),
                "theme": adventure.reasoning,
                "rating": float(rating) if rating else 0.0,
                "completion_status": completion_status,
                "timestamp": current_time.isoformat(),
                "date": current_time.strftime('%Y-%m-%d'),
                "day_of_week": current_time.strftime('%A'),
                "hour": current_time.hour,
                "season": self._get_season(current_time),
                "weather_preference": params.weather_preference,
                "constraints": ','.join(params.constraints) if params.constraints else ""
            }

            # Store in collection
            self.history_collection.add(
                ids=[adventure_id],
                documents=[adventure_doc],
                metadatas=[metadata]
            )

            # Update user preference profile
            self._update_user_preference_profile(normalized_user_id, adventure, params, rating)

            # Update location context
            self._update_location_context(normalized_user_id, params.location, adventure)

            print(f"✅ Stored adventure: {adventure.title} for user {normalized_user_id} (Rating: {rating}/5)")
            print(f"   Adventure ID: {adventure_id}")

            return adventure_id

        except Exception as e:
            print(f"⚠️ Error storing adventure: {e}")
            import traceback
            traceback.print_exc()
            return None

    def _get_season(self, date_obj: datetime) -> str:
        """Determine season from date"""
        month = date_obj.month
        if month in [12, 1, 2]:
            return "winter"
        elif month in [3, 4, 5]:
            return "spring"
        elif month in [6, 7, 8]:
            return "summer"
        else:
            return "fall"

    def _update_user_preference_profile(self, user_id: str, adventure: 'AdventureOption',
                                      params: 'AdventureParams', rating: float):
        """Update aggregated user preference profile"""

        try:
            profile_id = f"{user_id}_profile"

            # Try to get existing profile
            try:
                existing_results = self.profile_collection.get(
                    ids=[profile_id]
                )

                if existing_results['ids']:
                    # Update existing profile
                    existing_metadata = existing_results['metadatas'][0]

                    # Aggregate preferences
                    existing_prefs = existing_metadata.get('top_preferences', '').split(',')
                    all_prefs = existing_prefs + params.preferences
                    pref_counts = Counter(p.strip() for p in all_prefs if p.strip())
                    top_prefs = [pref for pref, _ in pref_counts.most_common(10)]

                    # Update aggregated data
                    total_adventures = existing_metadata.get('total_adventures', 0) + 1
                    total_rating_points = existing_metadata.get('total_rating_points', 0.0) + (rating or 0.0)
                    avg_rating = total_rating_points / total_adventures if total_adventures > 0 else 0.0

                    # Update mood patterns
                    mood_history = existing_metadata.get('mood_history', '').split(',')
                    mood_history.append(params.mood)
                    mood_counts = Counter(m.strip() for m in mood_history if m.strip())
                    favorite_mood = mood_counts.most_common(1)[0][0] if mood_counts else params.mood

                    # Budget patterns
                    budget_sum = existing_metadata.get('budget_sum', 0.0) + params.budget
                    avg_budget = budget_sum / total_adventures

                    # Time patterns
                    time_sum = existing_metadata.get('time_sum', 0) + params.time_available
                    avg_time = time_sum / total_adventures

                    updated_metadata = {
                        "user_id": user_id,
                        "total_adventures": total_adventures,
                        "top_preferences": ','.join(top_prefs),
                        "favorite_mood": favorite_mood,
                        "mood_history": ','.join(mood_history[-20:]),  # Keep last 20
                        "avg_rating": round(avg_rating, 2),
                        "total_rating_points": total_rating_points,
                        "avg_budget": round(avg_budget, 2),
                        "budget_sum": budget_sum,
                        "avg_time": round(avg_time),
                        "time_sum": time_sum,
                        "last_adventure": adventure.title,
                        "last_rating": float(rating) if rating else 0.0,
                        "last_updated": datetime.now(pytz.timezone('America/New_York')).isoformat(),
                        "highly_rated_themes": existing_metadata.get('highly_rated_themes', ''),
                        "energy_patterns": existing_metadata.get('energy_patterns', '')
                    }

                    # Track highly rated themes
                    if rating and rating >= 4:
                        highly_rated = existing_metadata.get('highly_rated_themes', '').split(',')
                        highly_rated.append(adventure.reasoning)
                        theme_counts = Counter(t.strip() for t in highly_rated if t.strip())
                        top_themes = [theme for theme, _ in theme_counts.most_common(5)]
                        updated_metadata['highly_rated_themes'] = ','.join(top_themes)

                    # Update the profile
                    self.profile_collection.update(
                        ids=[profile_id],
                        documents=[self._create_profile_document(updated_metadata)],
                        metadatas=[updated_metadata]
                    )

                    print(f"✅ Updated preference profile for {user_id}")

                else:
                    raise Exception("Profile not found")

            except:
                # Create new profile
                new_metadata = {
                    "user_id": user_id,
                    "total_adventures": 1,
                    "top_preferences": ','.join(params.preferences),
                    "favorite_mood": params.mood,
                    "mood_history": params.mood,
                    "avg_rating": float(rating) if rating else 0.0,
                    "total_rating_points": float(rating) if rating else 0.0,
                    "avg_budget": float(params.budget),
                    "budget_sum": float(params.budget),
                    "avg_time": int(params.time_available),
                    "time_sum": int(params.time_available),
                    "last_adventure": adventure.title,
                    "last_rating": float(rating) if rating else 0.0,
                    "last_updated": datetime.now(pytz.timezone('America/New_York')).isoformat(),
                    "highly_rated_themes": adventure.reasoning if rating and rating >= 4 else "",
                    "energy_patterns": params.energy_level
                }

                self.profile_collection.add(
                    ids=[profile_id],
                    documents=[self._create_profile_document(new_metadata)],
                    metadatas=[new_metadata]
                )

                print(f"✅ Created new preference profile for {user_id}")

        except Exception as e:
            print(f"⚠️ Error updating preference profile: {e}")

    def _create_profile_document(self, metadata: dict) -> str:
        """Create searchable document from profile metadata"""
        doc = f"""
        User Profile: {metadata['user_id']}
        Total Adventures: {metadata['total_adventures']}
        Top Preferences: {metadata['top_preferences']}
        Favorite Mood: {metadata['favorite_mood']}
        Average Rating: {metadata['avg_rating']}/5
        Average Budget: ${metadata['avg_budget']}
        Average Duration: {metadata['avg_time']} minutes
        Highly Rated Themes: {metadata.get('highly_rated_themes', '')}
        Energy Patterns: {metadata.get('energy_patterns', '')}
        """
        return doc

    def _update_location_context(self, user_id: str, location: str, adventure: 'AdventureOption'):
        """Track user's location patterns and preferences"""

        try:
            location_id = f"{user_id}_location_{hashlib.md5(location.encode()).hexdigest()[:8]}"

            location_doc = f"""
            User: {user_id}
            Location: {location}
            Adventure: {adventure.title}
            Theme: {adventure.reasoning}
            Steps: {'; '.join([step['activity'] for step in adventure.steps])}
            """

            location_metadata = {
                "user_id": user_id,
                "location": location,
                "adventure_title": adventure.title,
                "theme": adventure.reasoning,
                "timestamp": datetime.now(pytz.timezone('America/New_York')).isoformat()
            }

            self.location_collection.add(
                ids=[location_id],
                documents=[location_doc],
                metadatas=[location_metadata]
            )

            print(f"✅ Updated location context for {user_id} in {location}")

        except Exception as e:
            print(f"⚠️ Error updating location context: {e}")

    def get_user_preferences(self, user_id: str, n_results: int = 10) -> Dict:
        """ENHANCED: Get user's comprehensive preferences and history"""

        try:
            normalized_user_id = self._normalize_user_id(user_id)
            print(f"\n🔍 ENHANCED PROFILE LOOKUP:")
            print(f"   Original ID: '{user_id}'")
            print(f"   Normalized ID: '{normalized_user_id}'")

            # First, try to get the aggregated profile
            profile_id = f"{normalized_user_id}_profile"

            try:
                profile_results = self.profile_collection.get(
                    ids=[profile_id]
                )

                if profile_results['ids']:
                    profile_metadata = profile_results['metadatas'][0]

                    print(f"   ✅ Found aggregated profile:")
                    print(f"      Total adventures: {profile_metadata['total_adventures']}")
                    print(f"      Avg rating: {profile_metadata['avg_rating']}")
                    print(f"      Top preferences: {profile_metadata['top_preferences']}")

                    # Return enhanced profile data
                    return {
                        "has_history": True,
                        "user_id": normalized_user_id,
                        "total_adventures": profile_metadata['total_adventures'],
                        "common_preferences": [p.strip() for p in profile_metadata['top_preferences'].split(',') if p.strip()],
                        "favorite_mood": profile_metadata['favorite_mood'],
                        "average_rating": profile_metadata['avg_rating'],
                        "average_budget": profile_metadata['avg_budget'],
                        "average_time": profile_metadata['avg_time'],
                        "last_adventure": profile_metadata['last_adventure'],
                        "last_rating": profile_metadata['last_rating'],
                        "highly_rated_themes": [t.strip() for t in profile_metadata.get('highly_rated_themes', '').split(',') if t.strip()],
                        "energy_patterns": profile_metadata.get('energy_patterns', 'medium'),
                        "last_updated": profile_metadata['last_updated']
                    }

            except Exception as e:
                print(f"   ⚠️ Profile lookup error: {e}")

            # Fallback: Query individual adventure history
            print(f"   🔄 Falling back to individual adventure lookup...")

            results = self.history_collection.query(
                query_texts=[f"User {normalized_user_id} adventures preferences history"],
                where={"user_id": normalized_user_id},
                n_results=n_results
            )

            if not results['documents'] or not results['documents'][0]:
                print(f"   📝 No history found for user {normalized_user_id}")
                return {
                    "has_history": False,
                    "user_id": normalized_user_id,
                    "adventures": [],
                    "common_preferences": [],
                    "average_budget": 40.0,
                    "average_time": 120,
                    "favorite_mood": "exploratory",
                    "average_rating": 0.0
                }

            # Process individual adventures to build profile
            all_prefs = []
            budgets = []
            times = []
            moods = []
            ratings = []
            themes = []

            for metadata in results['metadatas'][0]:
                if metadata.get('preferences'):
                    all_prefs.extend(metadata['preferences'].split(','))
                if metadata.get('budget'):
                    budgets.append(metadata['budget'])
                if metadata.get('time_requested'):
                    times.append(metadata['time_requested'])
                if metadata.get('mood'):
                    moods.append(metadata['mood'])
                if metadata.get('rating') and metadata['rating'] > 0:
                    ratings.append(metadata['rating'])
                if metadata.get('theme'):
                    themes.append(metadata['theme'])

            # Analyze patterns
            pref_counts = Counter(p.strip() for p in all_prefs if p.strip())
            common_prefs = [pref for pref, count in pref_counts.most_common(5)]

            avg_budget = sum(budgets) / len(budgets) if budgets else 40.0
            avg_time = sum(times) / len(times) if times else 120
            favorite_mood = Counter(moods).most_common(1)[0][0] if moods else "exploratory"
            avg_rating = sum(ratings) / len(ratings) if ratings else 0.0

            print(f"   ✅ Built profile from {len(results['documents'][0])} adventures:")
            print(f"      Common preferences: {common_prefs}")
            print(f"      Average budget: ${avg_budget:.2f}")
            print(f"      Favorite mood: {favorite_mood}")
            print(f"      Average rating: {avg_rating:.1f}/5")

            return {
                "has_history": True,
                "user_id": normalized_user_id,
                "adventures": results['documents'][0],
                "common_preferences": common_prefs,
                "average_budget": avg_budget,
                "average_time": avg_time,
                "favorite_mood": favorite_mood,
                "average_rating": avg_rating,
                "highly_rated_themes": [theme for theme in set(themes) if themes.count(theme) >= 2],
                "total_adventures": len(results['documents'][0])
            }

        except Exception as e:
            print(f"⚠️ Error retrieving user preferences: {e}")
            import traceback
            traceback.print_exc()
            return {
                "has_history": False,
                "user_id": self._normalize_user_id(user_id),
                "adventures": [],
                "common_preferences": [],
                "average_budget": 40.0,
                "average_time": 120,
                "favorite_mood": "exploratory",
                "average_rating": 0.0
            }

    def get_relevant_tips(self, query: str, preferences: List[str], n_results: int = 3) -> List[Dict]:
        """Get relevant Boston tips with enhanced context matching"""

        try:
            # Enhanced search text with user context
            search_text = f"{query}. User interests: {', '.join(preferences)}. Boston local recommendations."

            print(f"\n🔍 ENHANCED RAG RETRIEVAL:")
            print(f"   Query: {search_text[:100]}...")

            results = self.tips_collection.query(
                query_texts=[search_text],
                n_results=n_results
            )

            if not results['documents'] or not results['documents'][0]:
                print(f"   ⚠️ No tips found")
                return []

            tips = []
            for i, doc in enumerate(results['documents'][0]):
                tip = {
                    "tip": doc,
                    "metadata": results['metadatas'][0][i] if results['metadatas'] else {},
                    "relevance": 1.0 - results['distances'][0][i] if results['distances'] else 0.0
                }
                tips.append(tip)
                print(f"   ✅ Tip {i+1}: {doc[:80]}... (relevance: {tip['relevance']:.2f})")

            return tips

        except Exception as e:
            print(f"⚠️ RAG retrieval error: {e}")
            return []

    def get_user_session_insights(self, user_id: str) -> Dict:
        """NEW: Get insights about user's current session"""

        normalized_user_id = self._normalize_user_id(user_id)
        session_data = self.current_session_data.get(normalized_user_id, {})

        return {
            "requests_this_session": session_data.get("requests", 0),
            "preferences_mentioned": session_data.get("preferences", []),
            "locations_explored": session_data.get("locations", []),
            "session_start": session_data.get("start_time")
        }

    def track_session_activity(self, user_id: str, activity_type: str, data: Dict):
        """NEW: Track user's session activity"""

        normalized_user_id = self._normalize_user_id(user_id)

        if normalized_user_id not in self.current_session_data:
            self.current_session_data[normalized_user_id] = {
                "start_time": datetime.now(pytz.timezone('America/New_York')).isoformat(),
                "requests": 0,
                "preferences": [],
                "locations": [],
                "activities": []
            }

        session = self.current_session_data[normalized_user_id]
        session["activities"].append({
            "type": activity_type,
            "data": data,
            "timestamp": datetime.now(pytz.timezone('America/New_York')).isoformat()
        })

        if activity_type == "request":
            session["requests"] += 1
        elif activity_type == "preferences":
            session["preferences"].extend(data.get("preferences", []))
        elif activity_type == "location":
            if data.get("location") not in session["locations"]:
                session["locations"].append(data.get("location"))

    def cleanup_old_data(self, days_threshold: int = 365):
        """NEW: Clean up old user data (GDPR compliance)"""

        try:
            cutoff_date = datetime.now(pytz.timezone('America/New_York')) - timedelta(days=days_threshold)
            cutoff_iso = cutoff_date.isoformat()

            # Get old adventures
            all_adventures = self.history_collection.get()
            old_ids = []

            for i, metadata in enumerate(all_adventures['metadatas']):
                if metadata.get('timestamp', '') < cutoff_iso:
                    old_ids.append(all_adventures['ids'][i])

            if old_ids:
                self.history_collection.delete(ids=old_ids)
                print(f"✅ Cleaned up {len(old_ids)} old adventures (older than {days_threshold} days)")

        except Exception as e:
            print(f"⚠️ Error cleaning up old data: {e}")

    def get_user_recommendations(self, user_id: str, context: Dict = None) -> Dict:
        """NEW: Get personalized recommendations based on user history"""

        profile = self.get_user_preferences(user_id)

        if not profile['has_history']:
            return {
                "recommendations": ["Try different neighborhoods", "Vary your time constraints", "Explore new activity types"],
                "confidence": "low"
            }

        recommendations = []

        # Analyze patterns
        if profile['average_rating'] < 3.5:
            recommendations.append(f"Consider trying {', '.join(profile['common_preferences'][:2])} activities - they seem to match your interests")

        if len(profile['common_preferences']) < 3:
            recommendations.append("Try adding more varied preferences to discover new experiences")

        # Time-based recommendations
        if profile.get('average_time', 120) < 60:
            recommendations.append("Consider longer adventures (90+ minutes) for more immersive experiences")

        # Budget-based recommendations
        if profile['average_budget'] < 25:
            recommendations.append("Many of Boston's best experiences are free - ask about budget-friendly options")

        # Theme recommendations
        highly_rated = profile.get('highly_rated_themes', [])
        if highly_rated:
            recommendations.append(f"You seem to love {highly_rated[0]} themes - try variations of this style")

        return {
            "recommendations": recommendations,
            "confidence": "high" if profile['total_adventures'] >= 3 else "medium"
        }

    def get_similar_users(self, user_id: str, n_results: int = 3) -> List[Dict]:
        """NEW: Find users with similar preferences for collaborative filtering"""

        profile = self.get_user_preferences(user_id)

        if not profile['has_history']:
            return []

        try:
            # Search for users with similar preferences
            search_text = f"User with preferences: {', '.join(profile['common_preferences'])}"

            results = self.profile_collection.query(
                query_texts=[search_text],
                where={"user_id": {"$ne": profile['user_id']}},  # Exclude current user
                n_results=n_results
            )

            similar_users = []
            for i, metadata in enumerate(results['metadatas'][0] if results['metadatas'] else []):
                similar_users.append({
                    "user_id": metadata['user_id'],
                    "common_preferences": metadata['top_preferences'].split(','),
                    "avg_rating": metadata['avg_rating'],
                    "total_adventures": metadata['total_adventures'],
                    "similarity": 1.0 - results['distances'][0][i] if results['distances'] else 0.0
                })

            print(f"✅ Found {len(similar_users)} similar users")
            return similar_users

        except Exception as e:
            print(f"⚠️ Error finding similar users: {e}")
            return []

    def get_trending_preferences(self, days: int = 30) -> Dict:
        """NEW: Get trending preferences across all users"""

        try:
            cutoff_date = datetime.now(pytz.timezone('America/New_York')) - timedelta(days=days)
            cutoff_iso = cutoff_date.isoformat()

            # Get recent adventures
            recent_results = self.history_collection.query(
                query_texts=["recent user preferences trends"],
                n_results=100
            )

            if not recent_results['metadatas']:
                return {"trending": [], "popular_themes": []}

            recent_prefs = []
            recent_themes = []
            recent_moods = []

            for metadata in recent_results['metadatas'][0]:
                if metadata.get('timestamp', '') >= cutoff_iso:
                    if metadata.get('preferences'):
                        recent_prefs.extend(metadata['preferences'].split(','))
                    if metadata.get('theme'):
                        recent_themes.append(metadata['theme'])
                    if metadata.get('mood'):
                        recent_moods.append(metadata['mood'])

            # Count trends
            pref_counts = Counter(p.strip() for p in recent_prefs if p.strip())
            theme_counts = Counter(recent_themes)
            mood_counts = Counter(recent_moods)

            return {
                "trending_preferences": [pref for pref, _ in pref_counts.most_common(5)],
                "popular_themes": [theme for theme, _ in theme_counts.most_common(3)],
                "trending_moods": [mood for mood, _ in mood_counts.most_common(3)],
                "data_period": f"Last {days} days"
            }

        except Exception as e:
            print(f"⚠️ Error getting trends: {e}")
            return {"trending": [], "popular_themes": []}

    def export_user_data(self, user_id: str) -> Dict:
        """NEW: Export all user data (GDPR compliance)"""

        normalized_user_id = self._normalize_user_id(user_id)

        try:
            # Get all user adventures
            adventures = self.history_collection.query(
                query_texts=["user data"],
                where={"user_id": normalized_user_id},
                n_results=1000
            )

            # Get user profile
            profile = self.get_user_preferences(normalized_user_id)

            export_data = {
                "user_id": normalized_user_id,
                "export_timestamp": datetime.now(pytz.timezone('America/New_York')).isoformat(),
                "profile": profile,
                "adventures": adventures,
                "total_records": len(adventures.get('documents', [[]])[0]) if adventures.get('documents') else 0
            }

            print(f"✅ Exported data for user {normalized_user_id}: {export_data['total_records']} records")
            return export_data

        except Exception as e:
            print(f"⚠️ Error exporting user data: {e}")
            return {"error": str(e)}

# Initialize the enhanced RAG system
rag_system = MiniQuestRAGSystem()
print("✅ Enhanced RAG System ready with robust user tracking!")

✅ Seeded 3 Boston insider tips
✅ Enhanced RAG system initialized with 4 collections
✅ Enhanced RAG System ready with robust user tracking!


In [ ]:
# Cell 5

# Model Router - OpenAI + Claude
class TaskType(Enum):
    PARSING = "parsing"
    REASONING = "reasoning"
    CREATIVE = "creative"
    OPTIMIZATION = "optimization"
    VALIDATION = "validation"

class ModelRouter:
    """Intelligent routing system - OpenAI + Claude"""

    def __init__(self):
        # Updated model map with Claude options
        self.model_map = {
            TaskType.PARSING: "gpt-4o-mini",                      # Fast parsing
            TaskType.REASONING: "claude-sonnet-4-20250514",       # Claude for reasoning
            TaskType.CREATIVE: "gpt-4o",                          # GPT-4 for creative
            TaskType.OPTIMIZATION: "claude-sonnet-4-20250514",    # Claude for optimization
            TaskType.VALIDATION: "gpt-4o-mini"                    # Fast validation
        }
        self.usage_stats = {model: 0 for model in set(self.model_map.values())}
        self.trace_log = []

    def route(self, task_type: TaskType, prompt: str,
              temperature: float = 0.7, max_tokens: int = 1000,
              trace_stage: str = None) -> str:
        """Route request to appropriate LLM (OpenAI or Claude)"""

        model_name = self.model_map[task_type]
        self.usage_stats[model_name] += 1

        print(f"🤖 Routing {task_type.value} to {model_name}")

        trace = {
            "stage": trace_stage or task_type.value,
            "model": model_name,
            "task": task_type.value,
            "prompt": prompt,
            "response": None,
            "timestamp": datetime.now(pytz.timezone('America/New_York')).isoformat()
        }

        try:
            # Route to Claude
            if "claude" in model_name:
                response = anthropic_client.messages.create(
                    model=model_name,
                    max_tokens=max_tokens,
                    temperature=temperature,
                    messages=[{"role": "user", "content": prompt}]
                )
                result = response.content[0].text

            # Route to OpenAI
            else:
                response = openai.chat.completions.create(
                    model=model_name,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=temperature,
                    max_tokens=max_tokens
                )
                result = response.choices[0].message.content

            trace["response"] = result
            self.trace_log.append(trace)

            print(f"✅ Response preview: {result[:100]}...")
            return result

        except Exception as e:
            print(f"❌ Error in {model_name}: {e}")
            trace["response"] = f"ERROR: {str(e)}"
            self.trace_log.append(trace)
            raise

    def get_stats(self) -> Dict:
        return {
            "total_calls": sum(self.usage_stats.values()),
            "by_model": self.usage_stats
        }

    def get_traces(self) -> List[Dict]:
        return self.trace_log

    def clear_traces(self):
        self.trace_log = []

# Initialize router
router = ModelRouter()
print("✅ Model router initialized (OpenAI + Claude)")

✅ Model router initialized (OpenAI + Claude)


In [ ]:
# Cell 6

# Location Detector
class LocationDetector:
    """Location detector"""

    def __init__(self, google_maps_key: str):
        self.maps_key = google_maps_key
        self.last_detected = None

        try:
            import googlemaps
            self.gmaps = googlemaps.Client(key=google_maps_key)
            print("✅ Google Maps geocoding client initialized")
        except ImportError:
            import subprocess
            subprocess.check_call(['pip', 'install', '-q', 'googlemaps'])
            import googlemaps
            self.gmaps = googlemaps.Client(key=google_maps_key)
            print("✅ Google Maps geocoding client initialized")

    def geocode_address(self, address: str) -> Dict:
        """Convert address to coordinates"""
        try:
            print(f"🏠 Geocoding address: {address}")

            geocode_result = self.gmaps.geocode(address)

            if not geocode_result:
                print(f"⚠️ No results found for address: {address}")
                return None

            result = geocode_result[0]
            location = result['geometry']['location']
            lat = location['lat']
            lon = location['lng']
            formatted_address = result['formatted_address']

            # Extract city, state from address components
            address_components = result['address_components']
            city = None
            state = None

            for component in address_components:
                types = component['types']
                if 'locality' in types:
                    city = component['long_name']
                elif 'administrative_area_level_1' in types:
                    state = component['short_name']

            location_parts = []
            if city:
                location_parts.append(city)
            if state:
                location_parts.append(state)

            clean_location = ", ".join(location_parts) if location_parts else formatted_address

            location_data = {
                "address": address,
                "formatted_address": formatted_address,
                "clean_location": clean_location,
                "search_location": clean_location,
                "lat": lat,
                "lon": lon,
                "timestamp": datetime.now(pytz.timezone('America/New_York')).isoformat(),
                "source": "address_geocoded",
            }

            print(f"✅ Geocoded: {address} → {clean_location} ({lat:.4f}, {lon:.4f})")
            self.last_detected = location_data
            return location_data

        except Exception as e:
            print(f"⚠️ Address geocoding error: {e}")
            return None

    def get_location_for_search(self, address_input: str = None, lat: float = None, lon: float = None) -> tuple:
        """Get the best location data for POI searches"""

        location_data = None

        if address_input and address_input.strip():
            location_data = self.geocode_address(address_input.strip())
        elif lat is not None and lon is not None:
            # Use coordinates directly
            location_data = {
                "clean_location": f"Location at {lat:.4f}, {lon:.4f}",
                "search_location": f"Location at {lat:.4f}, {lon:.4f}",
                "lat": lat,
                "lon": lon,
                "source": "coordinates"
            }
        elif self.last_detected:
            location_data = self.last_detected

        if location_data:
            return (
                location_data['clean_location'],
                location_data['search_location'],
                (location_data['lat'], location_data['lon']),
                location_data
            )
        else:
            return None, None, None, None

# Initialize location detector
location_detector = LocationDetector(GOOGLE_MAPS_KEY)
print("✅ Location Detector ready")

✅ Google Maps geocoding client initialized
✅ Location Detector ready


In [ ]:
# Cell 7
# Google Maps API with Distance Controls

class GoogleMapsAPI:
    """Google Maps API with smart distance filtering and time-aware radius"""

    def __init__(self, api_key: str):
        try:
            import googlemaps
            self.gmaps = googlemaps.Client(key=api_key)
            print("✅ Google Maps API initialized")
        except ImportError:
            import subprocess
            subprocess.check_call(['pip', 'install', '-q', 'googlemaps'])
            import googlemaps
            self.gmaps = googlemaps.Client(key=api_key)
            print("✅ Google Maps API initialized")

    def nearby_search(self, location: str, preferences: List[str],
                 exact_coords: Optional[tuple] = None,
                 radius: int = 3000, mood: str = None,
                 time_available: int = None) -> List[POI]:
        """Strict distance filtering and proper API calls"""

        try:

            if time_available:
                if time_available <= 10:
                    radius = 800   # 0.8km for 10 min
                    max_distance_km = 0.6  # Max 600m away - walkable in 7-8 minutes
                elif time_available <= 20:
                    radius = 1200  # 1.2km for 20 min
                    max_distance_km = 1.0  # Max 1km away
                elif time_available <= 30:
                    radius = 1500  # 1.5km for 30 min
                    max_distance_km = 1.5  # Max 1.5km away
                elif time_available <= 60:
                    radius = 2000  # 2km for 1 hour
                    max_distance_km = 2.5  # Max 2.5km away
                else:
                    radius = 3000  # 3km for longer trips
                    max_distance_km = 5.0  # Max 5km away
            else:
                max_distance_km = 2.0  # Conservative default

            print(f"🎯 STRICT filtering: {radius}m radius, max {max_distance_km}km for {time_available}min trip")

            # Set up location with error handling
            if exact_coords:
                lat, lon = exact_coords
                lat_lng = {"lat": lat, "lng": lon}
                print(f"🎯 Using exact coordinates: ({lat:.4f}, {lon:.4f}), radius: {radius}m")
            else:
                try:
                    # Error handling for geocoding
                    print(f"🔍 Geocoding location: '{location}'")
                    geocode_result = self.gmaps.geocode(location)
                    if not geocode_result:
                        print("⚠️ Geocoding failed, using fallback POIs")
                        return self._get_nearby_fallback_pois(max_distance_km)

                    lat_lng = geocode_result[0]['geometry']['location']
                    lat, lon = lat_lng['lat'], lat_lng['lng']
                    print(f"✅ Geocoded '{location}' to ({lat:.4f}, {lon:.4f})")
                except Exception as e:
                    print(f"❌ Geocoding error: {e}")
                    print("🔄 Using fallback POIs")
                    return self._get_nearby_fallback_pois(max_distance_km)

            all_pois = []
            seen_place_ids = set()

            # Generate location-specific search queries
            search_queries = self._generate_search_queries_with_llm(preferences, mood)
            print(f"🔍 Search queries: {search_queries}")

            # Execute searches with strict radius
            for query in search_queries[:3]:  # Limit to 3 queries for efficiency
                try:
                    print(f"📝 Searching: '{query}' within {radius}m")

                    # Use nearby_search instead of places() for better radius control
                    results = self.gmaps.places_nearby(
                        location=lat_lng,
                        radius=radius,  # Strict radius enforcement
                        keyword=query,
                        type='point_of_interest'
                    )

                    print(f"   📍 Found {len(results.get('results', []))} initial results")

                    for place in results.get('results', [])[:6]:  # Limit per query
                        place_id = place.get('place_id')
                        if not place_id or place_id in seen_place_ids:
                            continue

                        seen_place_ids.add(place_id)

                        # Calculate distance BEFORE getting details
                        place_lat = place.get('geometry', {}).get('location', {}).get('lat')
                        place_lon = place.get('geometry', {}).get('location', {}).get('lng')

                        if place_lat and place_lon:
                            distance_km = self._haversine_distance(lat, lon, place_lat, place_lon)

                            # STRICT distance filtering - reject early
                            if distance_km > max_distance_km:
                                place_name = place.get('name', 'Unknown')
                                print(f"   🚫 REJECTED: {place_name} - {distance_km:.1f}km away (too far for {time_available}min)")
                                continue

                            print(f"   ✅ ACCEPTED: {place.get('name', 'Unknown')} - {distance_km:.1f}km away")

                            # Now get detailed info for accepted places
                            details = self._get_place_details_safe(place_id, distance_km)
                            if details:
                                all_pois.append(details)

                except Exception as e:
                    print(f"⚠️ Search error for '{query}': {e}")
                    continue

            # Remove duplicates
            unique_pois = {}
            for poi in all_pois:
                if poi.name not in unique_pois:
                    unique_pois[poi.name] = poi

            final_pois = list(unique_pois.values())

            # Add nearby fallback only if needed and within distance
            if len(final_pois) < 2:
                print(f"⚠️ Only found {len(final_pois)} POIs, adding nearby fallbacks...")
                fallback_pois = self._get_nearby_fallback_pois(max_distance_km)
                for fallback_poi in fallback_pois:
                    if fallback_poi.name not in [p.name for p in final_pois]:
                        if fallback_poi.distance_from_user <= max_distance_km:
                            print(f"   ✅ Added nearby fallback: {fallback_poi.name} ({fallback_poi.distance_from_user:.1f}km)")
                            final_pois.append(fallback_poi)
                            if len(final_pois) >= 5:
                                break

            # Sort by distance for short trips
            if time_available and time_available <= 30:
                final_pois.sort(key=lambda x: x.distance_from_user or 999)
                print(f"   📍 Short trip: sorted by distance")
            else:
                final_pois.sort(key=lambda x: (x.distance_from_user * 0.3 + (5-x.rating) * 0.7))

            print(f"✅ FINAL RESULT: {len(final_pois)} locations within {max_distance_km}km")

            # Debug output
            for i, poi in enumerate(final_pois[:5], 1):
                distance_str = f"{poi.distance_from_user:.1f}km" if poi.distance_from_user else "unknown"
                print(f"   {i}. {poi.name} - {poi.rating}⭐ - {distance_str}")

            return final_pois[:8]  # Limit final results

        except Exception as e:
            print(f"⚠️ Search error: {e}")
            import traceback
            traceback.print_exc()
            return self._get_nearby_fallback_pois(1.0)  # 1km fallback

    def _get_place_details_safe(self, place_id: str, distance_km: float = None) -> Optional[POI]:
        """Safe place details with proper error handling"""
        try:
            details = self.gmaps.place(
                place_id=place_id,
                fields=[
                    'name', 'place_id', 'geometry', 'rating', 'price_level',
                    'editorial_summary', 'vicinity', 'formatted_address',
                    'user_ratings_total', 'business_status', 'reviews'
                ]
            )

            result = details.get('result', {})
            if not result:
                return None

            # Check business status
            if result.get('business_status') == 'CLOSED_PERMANENTLY':
                print(f"   ⚠️ Skipping permanently closed business: {result.get('name', 'Unknown')}")
                return None

            location = result.get('geometry', {}).get('location', {})
            lat = location.get('lat', 0)
            lon = location.get('lng', 0)

            if lat == 0 or lon == 0:
                print(f"   ⚠️ Invalid coordinates for {result.get('name', 'Unknown')}")
                return None

            name = result.get('name', 'Unknown Place')
            place_type = self._infer_place_type(name)
            rating = result.get('rating', 4.0)
            price_level = result.get('price_level', 2)

            description = result.get('editorial_summary', {}).get('overview', '')
            if not description:
                vicinity = result.get('vicinity', '')
                description = f"Popular {place_type}" + (f" in {vicinity}" if vicinity else "")

            tags = [place_type]
            if rating >= 4.5:
                tags.append("highly rated")

            # Extract reviews
            reviews_data = []
            raw_reviews = result.get('reviews', [])

            if raw_reviews:
                print(f"   📝 Found {len(raw_reviews)} reviews for {name}")
                for review in raw_reviews[:3]:  # Get top 3 reviews
                    reviews_data.append({
                        'author': review.get('author_name', 'Anonymous'),
                        'rating': review.get('rating', 0),
                        'text': review.get('text', ''),
                        'time': review.get('relative_time_description', 'Recently')
                    })

            poi = POI(
                id=place_id,
                name=name,
                type=place_type,
                description=description,
                lat=lat,
                lon=lon,
                rating=rating,
                price_level=price_level,
                current_wait=15,
                tags=tags,
                distance_from_user=distance_km,
                reviews=reviews_data
            )

            return poi

        except Exception as e:
            print(f"⚠️ Error getting place details for {place_id}: {e}")
            return None

    def _generate_search_queries_with_llm(self, preferences: List[str], mood: str = None) -> List[str]:
        """Use LLM to generate appropriate search queries based on user preferences"""

        # DEBUG: Print what we're actually receiving
        print(f"🔍 DEBUG - Input preferences: {preferences}")
        print(f"🔍 DEBUG - Input mood: {mood}")

        try:
            query_prompt = f"""Given user preferences and mood, generate 3-4 Google Places search terms.

    USER PREFERENCES: {', '.join(preferences)}
    USER MOOD: {mood or 'neutral'}

    Generate search terms that will find relevant businesses/locations on Google Maps.

    Examples:
    - Preferences: ["walking", "parks"], Mood: "relaxed" → ["parks", "walking trails", "gardens", "nature areas"]
    - Preferences: ["coffee", "bookstores"], Mood: "cozy" → ["coffee shops", "bookstores", "cafes", "libraries"]
    - Preferences: ["art", "galleries"], Mood: "cultural" → ["art galleries", "museums", "art centers", "exhibitions"]
    - Preferences: ["food", "restaurants"], Mood: "social" → ["restaurants", "dining", "food markets", "eateries"]

    Return ONLY a comma-separated list of 3-4 search terms, no other text."""

            # DEBUG: Print the prompt we're sending
            print(f"🔍 DEBUG - Prompt being sent to LLM:")
            print(f"   USER PREFERENCES: {', '.join(preferences)}")
            print(f"   USER MOOD: {mood or 'neutral'}")

            # Use the existing router to call OpenAI
            response = router.route(TaskType.PARSING, query_prompt, temperature=0.3)

            # DEBUG: Print the raw LLM response
            print(f"🔍 DEBUG - Raw LLM response: '{response}'")

            # Parse the response
            if response:
                # Clean up the response
                clean_response = response.strip().replace('"', '').replace('[', '').replace(']', '')
                queries = [q.strip() for q in clean_response.split(',')]

                # DEBUG: Print parsed queries
                print(f"🔍 DEBUG - Parsed queries: {queries}")

                # Filter and validate queries
                valid_queries = []
                for query in queries:
                    if query and len(query) > 2 and len(query) < 50:
                        valid_queries.append(query.lower())

                print(f"🔍 DEBUG - Final valid queries: {valid_queries}")

                if valid_queries:
                    return valid_queries[:4]  # Max 4 queries

            # Fallback if LLM fails
            print("⚠️ LLM query generation failed, using preference fallback")
            return self._get_fallback_queries(preferences, mood)

        except Exception as e:
            print(f"⚠️ LLM query generation error: {e}")
            return self._get_fallback_queries(preferences, mood)

    def _get_nearby_fallback_pois(self, max_distance_km: float) -> List[POI]:
        """Fallback with distance-appropriate Boston locations"""

        # Boston Common coordinates for reference
        boston_common_lat, boston_common_lon = 42.3555, -71.0640

        fallback_locations = [
            # Very close to downtown Boston
            POI("park1", "Boston Common", "park",
                "Historic central park perfect for quick walks",
                42.3555, -71.0640, 4.3, 0, 5, ["park", "historic"], distance_from_user=0.5),
            POI("park2", "Public Garden", "park",
                "Beautiful garden with Swan Boats",
                42.3541, -71.0711, 4.5, 0, 8, ["park", "garden"], distance_from_user=0.6),
            POI("cafe1", "Tatte Bakery & Cafe", "cafe",
                "European-style bakery with excellent coffee",
                42.3601, -71.0589, 4.5, 2, 10, ["coffee", "pastries"], distance_from_user=0.8),
        ]

        # Filter by distance
        nearby_pois = []
        for poi in fallback_locations:
            if poi.distance_from_user <= max_distance_km:
                nearby_pois.append(poi)

        print(f"   📍 Fallback: {len(nearby_pois)} locations within {max_distance_km}km")
        return nearby_pois

    def _get_fallback_queries(self, preferences: List[str], mood: str = None) -> List[str]:
        """Simple fallback when LLM fails"""

        print(f"🔍 DEBUG FALLBACK - Input preferences: {preferences}")

        fallback_queries = []

        for pref in preferences:
            pref_lower = pref.lower()
            print(f"🔍 DEBUG FALLBACK - Processing preference: '{pref_lower}'")

            if any(word in pref_lower for word in ['walk', 'park', 'nature', 'trail']):
                fallback_queries.extend(['parks', 'walking trails'])
                print(f"   → Added: parks, walking trails")
            elif any(word in pref_lower for word in ['coffee', 'cafe']):
                fallback_queries.extend(['coffee shops', 'cafes'])
                print(f"   → Added: coffee shops, cafes")
            elif any(word in pref_lower for word in ['art', 'gallery', 'museum']):
                fallback_queries.extend(['art galleries', 'museums'])
                print(f"   → Added: art galleries, museums")
            elif any(word in pref_lower for word in ['book', 'library']):
                fallback_queries.extend(['bookstores', 'libraries'])
                print(f"   → Added: bookstores, libraries")
            elif any(word in pref_lower for word in ['food', 'restaurant', 'eat']):
                fallback_queries.extend(['restaurants', 'food'])
                print(f"   → Added: restaurants, food")
            else:
                fallback_queries.append(pref_lower)
                print(f"   → Added generic: {pref_lower}")

        # Remove duplicates and return
        unique_queries = list(dict.fromkeys(fallback_queries))
        print(f"🔍 DEBUG FALLBACK - Final unique queries: {unique_queries}")

        return unique_queries[:4] if unique_queries else ['attractions', 'points of interest']

    def _infer_place_type(self, name: str) -> str:
        """Infer place type from name since we can't get it from API"""
        name_lower = name.lower()

        if any(word in name_lower for word in ['coffee', 'cafe', 'espresso', 'roast', 'brew']):
            return 'cafe'
        elif any(word in name_lower for word in ['book', 'library', 'read']):
            return 'bookstore'
        elif any(word in name_lower for word in ['restaurant', 'bistro', 'kitchen', 'grill', 'eatery']):
            return 'restaurant'
        elif any(word in name_lower for word in ['museum', 'gallery', 'art', 'exhibition']):
            return 'museum'
        elif any(word in name_lower for word in ['park', 'garden', 'green', 'common']):
            return 'park'
        elif any(word in name_lower for word in ['bar', 'pub', 'lounge', 'tavern']):
            return 'bar'
        else:
            return 'establishment'

    def _haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """Calculate distance in kilometers"""
        import math
        lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        return 6371 * c  # Earth's radius in kilometers

    def _get_fallback_pois(self) -> List[POI]:
        """Fallback with nearby Boston locations"""
        return [
            # Downtown Boston locations (close to city center)
            POI("park1", "Boston Common", "park",
                "Historic central park perfect for quick walks and relaxation",
                42.3555, -71.0640, 4.3, 0, 5, ["park", "historic", "walking"]),
            POI("park2", "Public Garden", "park",
                "Beautiful landscaped garden with iconic Swan Boats pond",
                42.3541, -71.0711, 4.5, 0, 8, ["park", "garden", "scenic"]),
            POI("park3", "Christopher Columbus Waterfront Park", "park",
                "Waterfront park with harbor views and green space",
                42.3650, -71.0470, 4.2, 0, 3, ["park", "waterfront", "views"]),
            POI("park4", "Norman B. Leventhal Park", "park",
                "Small downtown park perfect for a quick nature break",
                42.3519, -71.0552, 4.1, 0, 2, ["park", "downtown", "quiet"]),
            # Close-by cafes
            POI("cafe1", "Tatte Bakery & Cafe", "cafe",
                "European-style bakery with excellent coffee and pastries",
                42.3601, -71.0589, 4.5, 2, 10, ["coffee", "pastries"]),
            POI("cafe2", "Thinking Cup", "cafe",
                "Local coffee shop with cozy atmosphere",
                42.3573, -71.0636, 4.3, 2, 8, ["coffee", "local"]),
        ]

print("✅ Google Maps API with smart distance filtering")

✅ Google Maps API with smart distance filtering


In [ ]:
# Cell 8

# Weather API
class WeatherAPI:
    """Simple weather API using Weather.gov (free, no key needed)"""

    def __init__(self):
        self.base_url = "https://api.weather.gov"
        self.headers = {"User-Agent": "(MiniQuest Adventure App, contact@miniquest.app)"}

    def get_current_weather(self, location: str = None, lat: float = None, lon: float = None) -> Dict:
        """Get current weather conditions"""
        try:
            # Default to Boston if no location
            if not lat or not lon:
                lat, lon = 42.3601, -71.0589

            response = requests.get(f"{self.base_url}/points/{lat:.4f},{lon:.4f}",
                                   headers=self.headers, timeout=5)
            points_data = response.json()

            # Get latest observation
            if 'properties' in points_data:
                stations_url = points_data['properties']['observationStations']
                stations_response = requests.get(stations_url, headers=self.headers, timeout=5)
                stations = stations_response.json()

                if stations['features']:
                    station_id = stations['features'][0]['properties']['stationIdentifier']
                    obs_response = requests.get(
                        f"{self.base_url}/stations/{station_id}/observations/latest",
                        headers=self.headers, timeout=5
                    )
                    observation = obs_response.json()
                    props = observation['properties']

                    return {
                        "weather": [{"main": "Clear", "description": props.get('textDescription', 'clear')}],
                        "main": {
                            "temp": self._celsius_to_fahrenheit(props.get('temperature', {}).get('value', 20)),
                            "feels_like": self._celsius_to_fahrenheit(props.get('temperature', {}).get('value', 20))
                        },
                        "source": "weather.gov"
                    }

        except Exception as e:
            print(f"⚠️ Weather API error: {e}")

        # Fallback weather
        return {
            "weather": [{"main": "Clear", "description": "clear sky"}],
            "main": {"temp": 65, "feels_like": 63},
            "source": "fallback"
        }

    def _celsius_to_fahrenheit(self, celsius: float) -> float:
        if celsius is None:
            return 65.0
        return (celsius * 9/5) + 32

    def is_outdoor_friendly(self, weather_data: Dict) -> bool:
        """Check if weather is good for outdoor activities"""
        temp = weather_data['main']['temp']
        conditions = weather_data['weather'][0]['description'].lower()
        bad_weather = any(word in conditions for word in ['rain', 'snow', 'storm'])
        return not bad_weather and 40 < temp < 90

print("✅ Weather API defined")

✅ Weather API defined


In [ ]:
# Cell 8.1

class SunriseSunsetAPI:
    """Golden hour timing for photography and outdoor activity recommendations"""

    def __init__(self):
        self.base_url = "https://api.sunrise-sunset.org/json"
        print("✅ Sunrise/Sunset API initialized")

    def get_sun_times(self, lat: float, lon: float, date=None) -> Dict:
        """Get sunrise/sunset times for location"""
        try:
            params = {
                'lat': lat,
                'lng': lon,
                'formatted': 0  # Get UTC times
            }

            if date:
                params['date'] = date

            response = requests.get(self.base_url, params=params, timeout=5)
            data = response.json()

            if data['status'] == 'OK':
                results = data['results']

                # Convert to local timezone (Boston)
                boston_tz = pytz.timezone('America/New_York')

                def parse_utc_time(utc_string):
                    dt_utc = datetime.fromisoformat(utc_string.replace('Z', '+00:00'))
                    return dt_utc.astimezone(boston_tz)

                sunrise_local = parse_utc_time(results['sunrise'])
                sunset_local = parse_utc_time(results['sunset'])
                civil_begin_local = parse_utc_time(results['civil_twilight_begin'])
                civil_end_local = parse_utc_time(results['civil_twilight_end'])

                # Golden hour calculations
                golden_hour_morning_start = sunrise_local - timedelta(minutes=30)
                golden_hour_morning_end = sunrise_local + timedelta(minutes=30)
                golden_hour_evening_start = sunset_local - timedelta(minutes=30)
                golden_hour_evening_end = sunset_local + timedelta(minutes=30)

                return {
                    'sunrise': sunrise_local.strftime('%I:%M %p'),
                    'sunset': sunset_local.strftime('%I:%M %p'),
                    'civil_twilight_begin': civil_begin_local.strftime('%I:%M %p'),
                    'civil_twilight_end': civil_end_local.strftime('%I:%M %p'),
                    'golden_hour_morning': f"{golden_hour_morning_start.strftime('%I:%M %p')} - {golden_hour_morning_end.strftime('%I:%M %p')}",
                    'golden_hour_evening': f"{golden_hour_evening_start.strftime('%I:%M %p')} - {golden_hour_evening_end.strftime('%I:%M %p')}",
                    'sunrise_raw': sunrise_local,
                    'sunset_raw': sunset_local,
                    'golden_start_morning': golden_hour_morning_start,
                    'golden_end_morning': golden_hour_morning_end,
                    'golden_start_evening': golden_hour_evening_start,
                    'golden_end_evening': golden_hour_evening_end,
                    'source': 'sunrise-sunset.org',
                    'location': f"{lat:.4f}, {lon:.4f}"
                }
            else:
                raise Exception(f"API returned error: {data.get('status')}")

        except Exception as e:
            print(f"⚠️ Sunrise API error: {e}")
            # Fallback times for Boston area
            boston_tz = pytz.timezone('America/New_York')
            now = datetime.now(boston_tz)

            # Reasonable defaults based on season
            if now.month in [6, 7, 8]:  # Summer
                sunrise_time = "05:30 AM"
                sunset_time = "07:30 PM"
                golden_evening = "07:00 PM - 08:00 PM"
            elif now.month in [12, 1, 2]:  # Winter
                sunrise_time = "07:00 AM"
                sunset_time = "05:00 PM"
                golden_evening = "04:30 PM - 05:30 PM"
            else:  # Spring/Fall
                sunrise_time = "06:30 AM"
                sunset_time = "06:30 PM"
                golden_evening = "06:00 PM - 07:00 PM"

            return {
                'sunrise': sunrise_time,
                'sunset': sunset_time,
                'civil_twilight_begin': "05:30 AM",
                'civil_twilight_end': "08:00 PM",
                'golden_hour_morning': "06:00 AM - 07:00 AM",
                'golden_hour_evening': golden_evening,
                'source': 'fallback',
                'location': f"{lat:.4f}, {lon:.4f}"
            }

    def is_golden_hour(self, lat: float, lon: float) -> Dict:
        """Check if it's currently golden hour and return detailed info"""
        try:
            sun_times = self.get_sun_times(lat, lon)
            boston_tz = pytz.timezone('America/New_York')
            current_time = datetime.now(boston_tz)

            # Check if we have raw datetime objects
            if 'golden_start_morning' in sun_times and 'golden_end_morning' in sun_times:
                morning_golden = (sun_times['golden_start_morning'] <= current_time <= sun_times['golden_end_morning'])
                evening_golden = (sun_times['golden_start_evening'] <= current_time <= sun_times['golden_end_evening'])

                is_golden = morning_golden or evening_golden

                if morning_golden:
                    golden_type = "morning"
                    time_remaining = (sun_times['golden_end_morning'] - current_time).total_seconds() / 60
                elif evening_golden:
                    golden_type = "evening"
                    time_remaining = (sun_times['golden_end_evening'] - current_time).total_seconds() / 60
                else:
                    golden_type = None
                    # Calculate minutes until next golden hour
                    if current_time < sun_times['golden_start_morning']:
                        time_remaining = (sun_times['golden_start_morning'] - current_time).total_seconds() / 60
                    elif current_time < sun_times['golden_start_evening']:
                        time_remaining = (sun_times['golden_start_evening'] - current_time).total_seconds() / 60
                    else:
                        # Next golden hour is tomorrow morning
                        time_remaining = None

                return {
                    'is_golden_hour': is_golden,
                    'golden_type': golden_type,
                    'time_remaining_minutes': int(time_remaining) if time_remaining else None,
                    'current_time': current_time.strftime('%I:%M %p'),
                    'sun_times': sun_times,
                    'recommendation': self._get_golden_hour_recommendation(is_golden, golden_type, time_remaining)
                }

            else:
                # Fallback parsing
                return {
                    'is_golden_hour': False,
                    'golden_type': None,
                    'time_remaining_minutes': None,
                    'current_time': current_time.strftime('%I:%M %p'),
                    'sun_times': sun_times,
                    'recommendation': "Check sunset time for best photography lighting"
                }

        except Exception as e:
            print(f"⚠️ Golden hour check error: {e}")
            return {
                'is_golden_hour': False,
                'golden_type': None,
                'error': str(e),
                'recommendation': "Great lighting available throughout the day"
            }

    def _get_golden_hour_recommendation(self, is_golden: bool, golden_type: str, time_remaining: float) -> str:
        """Get recommendation based on golden hour status"""

        if is_golden:
            if golden_type == "morning":
                return f"🌅 Perfect morning golden hour! {int(time_remaining)} minutes of magical lighting remaining."
            else:
                return f"🌆 Beautiful evening golden hour! {int(time_remaining)} minutes of perfect photography light."

        elif time_remaining and time_remaining < 60:
            return f"🕐 Golden hour starts in {int(time_remaining)} minutes - perfect timing for outdoor activities!"

        elif time_remaining and time_remaining < 180:
            return f"⏰ Golden hour in {int(time_remaining//60)}h {int(time_remaining%60)}min - great for planning outdoor adventures!"

        else:
            return "🌤️ Good lighting available - perfect for exploring Boston!"

    def should_use_for_request(self, user_input: str, preferences: list) -> bool:
        """Determine if sunrise/sunset API should be used based on user request"""

        # Keywords that indicate user wants timing/photography features
        timing_keywords = [
            'sunrise', 'sunset', 'golden hour', 'photography', 'photo', 'pics', 'pictures',
            'lighting', 'best time', 'when to go', 'timing', 'golden light', 'magic hour',
            'blue hour', 'twilight', 'dawn', 'dusk', 'early morning', 'evening light'
        ]

        # Activity preferences that often benefit from timing
        timing_activities = [
            'photography', 'photos', 'scenic', 'views', 'outdoor', 'nature', 'park',
            'waterfront', 'harbor', 'rooftop', 'lookout', 'vista'
        ]

        text_to_check = f"{user_input.lower()} {' '.join(preferences).lower()}"

        # Direct keyword match
        for keyword in timing_keywords:
            if keyword in text_to_check:
                return True

        # Activity-based match
        for activity in timing_activities:
            if activity in text_to_check:
                # Only use if combined with timing-related context
                timing_context = ['best', 'perfect', 'ideal', 'when', 'time']
                if any(context in text_to_check for context in timing_context):
                    return True

        return False

    def get_photography_recommendations(self, lat: float, lon: float, activity_type: str = None) -> Dict:
        """Get photography and lighting recommendations - ONLY when specifically requested"""

        golden_status = self.is_golden_hour(lat, lon)
        sun_times = golden_status.get('sun_times', {})

        recommendations = []

        # Activity-specific recommendations
        if activity_type:
            if 'park' in activity_type.lower() or 'outdoor' in activity_type.lower():
                if golden_status['is_golden_hour']:
                    recommendations.append("🌅 Perfect for landscape photography with golden lighting!")
                    recommendations.append("📸 Great time for outdoor portraits with natural lighting")
                else:
                    recommendations.append(f"🌆 Best outdoor photos during golden hour: {sun_times.get('golden_hour_evening', 'Evening')}")

            elif 'coffee' in activity_type.lower() or 'cafe' in activity_type.lower():
                recommendations.append("☕ Window seats provide great natural lighting for cozy photos")
                if golden_status['is_golden_hour']:
                    recommendations.append("✨ Golden hour creates warm, inviting cafe atmosphere")

            elif 'museum' in activity_type.lower() or 'art' in activity_type.lower():
                recommendations.append("🖼️ Indoor lighting perfect regardless of outdoor conditions")
                recommendations.append("📱 Many museums allow photography - check first!")

        # General recommendations
        if not golden_status['is_golden_hour'] and sun_times.get('golden_hour_evening'):
            recommendations.append(f"🕖 Plan outdoor photos for {sun_times['golden_hour_evening']} for best lighting")

        return {
            'golden_status': golden_status,
            'recommendations': recommendations,
            'best_photo_time': sun_times.get('golden_hour_evening', 'Evening'),
            'sunrise_time': sun_times.get('sunrise'),
            'sunset_time': sun_times.get('sunset')
        }

# Initialize the Sunrise/Sunset API
sunrise_sunset_api = SunriseSunsetAPI()
print("✅ Sunrise/Sunset API ready for golden hour recommendations!")

✅ Sunrise/Sunset API initialized
✅ Sunrise/Sunset API ready for golden hour recommendations!


In [ ]:
# Cell 9

# Intent Parser - OpenAI + Claude
class ConversationalIntentParser:
    """Parse user input into structured parameters with accuracy"""

    def parse_with_clarification(self, user_input: str) -> tuple[Optional[AdventureParams], List[str]]:
        """Extract adventure parameters with better preference detection"""

        extraction_prompt = f"""You are an intelligent adventure planning assistant. Extract structured parameters from this user request:

USER REQUEST: "{user_input}"

Analyze the request and return ONLY a JSON object with these fields:

{{
  "mood": "user's emotional tone/vibe or null",
  "time_available": <total minutes as integer or null>,
  "budget": <total dollars as float or null>,
  "location": "city/area mentioned or null",
  "energy_level": "low/medium/high or null",
  "preferences": ["activity1", "activity2", ...],
  "constraints": ["limitation1", "limitation2", ...],
  "weather_preference": "indoor/outdoor/flexible or null"
}}

INTELLIGENT EXTRACTION GUIDELINES:

**Time Parsing:**
- "10 mins" → 10, "2 hours" → 120, "3 hours" → 180, "half hour" → 30
- "quick" → 30, "afternoon" → 180, "morning" → 120, "all day" → 360
- "few hours" → 150, "couple hours" → 120

**Budget Parsing:**
- "free" → 0.0, "$30" → 30.0, "$100" → 100.0
- "cheap" → 15.0, "moderate" → 40.0, "expensive" → 75.0, "luxury" → 150.0

**Mood Detection:**
- Look for emotional words: relaxed, cozy, energetic, adventurous, cultural, social, romantic, chill, excited
- Infer from context: "feel like shopping" → "retail therapy" or "browsing"
- "not hungry" → indicates NOT food-focused

**CRITICAL: Preferences Extraction (MOST IMPORTANT):**

You MUST extract preferences from BOTH explicit nouns AND implicit verbs/activities:

1. **From Activity Verbs** (convert verb → noun preferences):
   - "shopping" or "browse" or "buy" → ["shopping", "retail stores", "boutiques"]
   - "eating" or "hungry" → ["restaurants", "food", "dining"]
   - "walking" or "stroll" → ["walking", "parks", "trails"]
   - "drinking coffee" → ["coffee shops", "cafes"]
   - "looking at art" → ["art galleries", "museums"]

2. **From Explicit Nouns**:
   - "coffee shops" → ["coffee shops", "cafes"]
   - "bookstores" → ["bookstores", "libraries", "books"]
   - "parks" → ["parks", "gardens", "nature"]
   - "galleries" → ["art galleries", "museums", "art"]

3. **From Context Clues**:
   - "retail therapy" → ["shopping", "boutiques", "stores"]
   - "cultural exploration" → ["museums", "galleries", "historic sites"]
   - "foodie adventure" → ["restaurants", "food markets", "cafes"]

4. **Negative Indicators**:
   - "not hungry" → DO NOT include food/restaurant preferences
   - "avoid crowds" → Add to constraints, not preferences

**EXAMPLES WITH COMPLETE EXTRACTION:**

Input: "I feel like shopping something, 3 hours, $100"
Output: {{
  "mood": "retail therapy",
  "time_available": 180,
  "budget": 100.0,
  "preferences": ["shopping", "retail stores", "boutiques", "malls"],
  "location": null,
  "constraints": []
}}

Input: "I'm in Boston for 4 hours, make it memorable"
Output: {{
  "mood": "memorable experience",
  "time_available": 240,
  "budget": null,
  "preferences": ["sightseeing", "landmarks", "tourist attractions", "iconic Boston spots"],
  "location": "Boston"
}}

Input: "First time in the city, what should I see?"
Output: {{
  "mood": "exploratory",
  "time_available": null,
  "budget": null,
  "preferences": ["sightseeing", "landmarks", "must-see attractions"],
  "location": null
}}

Input: "Layover, want to remember this place"
Output: {{
  "mood": "memorable",
  "time_available": null,
  "budget": null,
  "preferences": ["tourist attractions", "iconic spots", "memorable experiences"],
  "location": null
}}

Input: "Coffee shops and bookstores, 90 minutes, $25, cozy"
Output: {{
  "mood": "cozy",
  "time_available": 90,
  "budget": 25.0,
  "preferences": ["coffee shops", "cafes", "bookstores", "books"],
  "location": null
}}

Input: "Want to browse art galleries, not hungry though"
Output: {{
  "mood": "cultural",
  "time_available": null,
  "budget": null,
  "preferences": ["art galleries", "museums", "art", "culture"],
  "constraints": ["no food venues"],
  "location": null
}}

Input: "Quick walk in parks, 45 min, free"
Output: {{
  "mood": "relaxed",
  "time_available": 45,
  "budget": 0.0,
  "preferences": ["parks", "walking trails", "gardens", "nature"],
  "location": null
}}

Input: "Live music tonight, $60, energetic"
Output: {{
  "mood": "energetic",
  "time_available": null,
  "budget": 60.0,
  "preferences": ["live music", "music venues", "concerts", "bars with music"],
  "location": null
}}

**CRITICAL RULES:**
1. If you find ANY activity verb, convert it to preference nouns
2. Preferences array should have 2-5 items (be generous but relevant)
3. If "not hungry" or similar → DO NOT add food preferences
4. Always infer mood from emotional context
5. Return ONLY valid JSON, no explanations

Return ONLY the JSON object, no other text."""

        response = router.route(TaskType.PARSING, extraction_prompt, temperature=0.4)

        try:
            # Clean response
            response = response.strip()
            if response.startswith("```json"):
                response = response.split("```json")[1].split("```")[0]
            elif response.startswith("```"):
                response = response.split("```")[1].split("```")[0]

            params_dict = json.loads(response)

            print(f"\n🔍 PARSING DEBUG:")
            print(f"   Input: '{user_input}'")
            print(f"   Raw Preferences: {params_dict.get('preferences')}")
            print(f"   Mood: {params_dict.get('mood')}")
            print(f"   Time: {params_dict.get('time_available')} min")
            print(f"   Budget: ${params_dict.get('budget')}")

            # CRITICAL: Check if preferences are missing or empty
            if not params_dict.get("preferences") or params_dict.get("preferences") == []:
                print(f"   ⚠️ WARNING: No preferences extracted from LLM!")
                print(f"   🔧 Running emergency inference...")
                inferred_prefs = self._emergency_preference_extraction(user_input)
                params_dict["preferences"] = inferred_prefs
                print(f"   ✅ Inferred preferences: {inferred_prefs}")

            # Validate preferences aren't too generic
            if params_dict.get("preferences") == ["exploration"]:
                print(f"   ⚠️ WARNING: Only generic 'exploration' found!")
                print(f"   🔧 Re-running inference...")
                better_prefs = self._emergency_preference_extraction(user_input)
                if better_prefs and better_prefs != ["exploration"]:
                    params_dict["preferences"] = better_prefs
                    print(f"   ✅ Better preferences: {better_prefs}")

            # Track missing fields
            missing = []
            if params_dict.get("time_available") is None:
                missing.append("time")
            if params_dict.get("budget") is None:
                missing.append("budget")
            if params_dict.get("location") is None:
                missing.append("location")

            # Fill reasonable defaults
            if not params_dict.get("mood"):
                params_dict["mood"] = "exploratory"
            if not params_dict.get("energy_level"):
                params_dict["energy_level"] = "medium"
            if not params_dict.get("weather_preference"):
                params_dict["weather_preference"] = "flexible"
            if not params_dict.get("constraints"):
                params_dict["constraints"] = []

            # Set reasonable defaults for missing required fields
            if params_dict.get("time_available") is None:
                params_dict["time_available"] = 120  # 2 hours default
            if params_dict.get("budget") is None:
                params_dict["budget"] = 40.0  # $40 default
            if not params_dict.get("location"):
                params_dict["location"] = "Boston, MA"

            params = AdventureParams(**params_dict)

            print(f"✅ Final parsed preferences: {params.preferences}")
            print(f"✅ Parsing successful\n")

            return params, missing

        except Exception as e:
            print(f"⚠️ Parsing error: {e}")
            print(f"Raw response: {response}")
            return None, ["parsing_failed"]

    def _emergency_preference_extraction(self, user_input: str) -> List[str]:
        """Emergency fallback with better pattern matching"""

        text = user_input.lower()
        preferences = []

        print(f"   🔍 Emergency extraction analyzing: '{text}'")

        # Activity mapping with verb detection
        activity_patterns = {
            # Shopping patterns
            'shop|shopping|browse|browsing|retail|buy|buying|mall|stores':
                ['shopping', 'retail stores', 'boutiques', 'malls'],

            # Food patterns (but check for negatives first)
            'eat|eating|hungry|food|restaurant|dining':
                ['restaurants', 'dining', 'food'],

            # Coffee patterns
            'coffee|cafe|espresso|latte|cappuccino':
                ['coffee shops', 'cafes'],

            # Nature patterns
            'park|parks|garden|nature|green|outdoor|walk|walking|stroll|hike|hiking':
                ['parks', 'walking trails', 'gardens', 'nature'],

            # Book patterns
            'book|bookstore|library|read|reading':
                ['bookstores', 'libraries', 'books'],

            # Art/Culture patterns
            'art|gallery|galleries|museum|museums|exhibit|culture|cultural':
                ['art galleries', 'museums', 'art', 'culture'],

            # Music patterns
            'music|concert|band|show|live music|performance':
                ['music venues', 'concerts', 'live music'],

            # Drinks/Nightlife patterns
            'bar|pub|drink|drinking|brewery|breweries|nightlife|cocktail':
                ['bars', 'breweries', 'nightlife', 'pubs'],

            # Fitness patterns
            'yoga|fitness|gym|workout|exercise':
                ['yoga studios', 'fitness', 'gyms'],

            # Entertainment patterns
            'movie|cinema|film|theater|theatre':
                ['movie theaters', 'entertainment', 'cinema']
        }

        # Check for negative indicators first
        negative_patterns = ['not hungry', 'no food', "don't want to eat", 'skip food', 'avoid eating']
        exclude_food = any(neg in text for neg in negative_patterns)

        if exclude_food:
            print(f"   🚫 Detected negative indicator: excluding food preferences")

        # Match patterns
        import re
        matched_categories = []

        for pattern, pref_list in activity_patterns.items():
            if re.search(pattern, text):
                # Skip food if negative indicator found
                if exclude_food and any(food_word in pattern for food_word in ['eat', 'food', 'restaurant', 'hungry']):
                    print(f"   🚫 Skipping food pattern due to negative indicator")
                    continue

                print(f"   ✓ Matched pattern: '{pattern}' → {pref_list[:2]}")
                preferences.extend(pref_list)
                matched_categories.append(pattern.split('|')[0])
                break  # Only take first strong match to avoid over-extraction

        # Remove duplicates while preserving order
        seen = set()
        unique_prefs = []
        for pref in preferences:
            if pref not in seen:
                seen.add(pref)
                unique_prefs.append(pref)

        # Final result
        final_prefs = unique_prefs[:4] if unique_prefs else ['exploration', 'local spots']

        print(f"   ✅ Emergency extraction result: {final_prefs}")

        return final_prefs

print("✅ Intent Parser defined")

✅ Intent Parser defined


In [ ]:
# Cell 10

# Photo API
class GooglePlacesPhotosAPI:
    """Fetch photos for POIs"""

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://maps.googleapis.com/maps/api/place/photo"

    def get_place_photo_url(self, photo_reference: str, max_width: int = 400) -> str:
        return f"{self.base_url}?maxwidth={max_width}&photo_reference={photo_reference}&key={self.api_key}"

    def get_poi_images(self, place_id: str, max_photos: int = 2) -> List[Dict]:
        """Get image URLs for a POI"""
        try:
            import googlemaps
            gmaps = googlemaps.Client(key=self.api_key)

            place_details = gmaps.place(place_id=place_id, fields=['photo', 'name'])

            photos = []
            if 'photos' in place_details.get('result', {}):
                for photo in place_details['result']['photos'][:max_photos]:
                    photo_ref = photo.get('photo_reference')
                    if photo_ref:
                        photos.append({
                            'url': self.get_place_photo_url(photo_ref, max_width=400),
                            'width': photo.get('width', 400),
                            'height': photo.get('height', 300)
                        })

            if photos:
                print(f"   ✅ Found {len(photos)} photos")

            return photos

        except Exception as e:
            print(f"⚠️ Error getting photos: {e}")
            return []

print("✅ Photo API defined")

✅ Photo API defined


In [ ]:
# Cell 11
# Scout Agent with Time-Aware Search

class ScoutAgent:
    """Scout agent to find and score POIs with RAG enhancement, time-aware distance filtering, and conditional sunrise/sunset API"""

    def __init__(self, maps_api, weather_api, photos_api, rag_system, sunrise_sunset_api=None):
        self.maps_api = maps_api
        self.weather_api = weather_api
        self.photos_api = photos_api
        self.rag_system = rag_system
        self.sunrise_sunset_api = sunrise_sunset_api  # Optional sunrise/sunset API

    def scout_with_images(self, params: AdventureParams, user_id: str = "demo_user",
                     address_input: str = None, lat: float = None, lon: float = None,
                     user_input: str = "") -> tuple[List[EnhancedPOI], Dict]:
        """Find POIs with images and RAG insider knowledge - Enhanced with conditional sunrise API"""

        # DEBUG: Print what we're receiving
        print(f"🔧 SCOUT DEBUG - Received parameters:")
        print(f"   address_input: '{address_input}'")
        print(f"   lat: {lat}")
        print(f"   lon: {lon}")
        print(f"   params.location: '{params.location}'")
        print(f"   user_input: '{user_input}'")

        # Coordinate and location string determination
        coords = None
        search_location = None
        display_location = None

        # Priority 1: Use manual address if provided
        if address_input and address_input.strip():
            search_location = address_input.strip()
            display_location = address_location = address_input.strip()
            params.location = search_location

            # If we also have coordinates, use them for precise searching
            if lat is not None and lon is not None:
                coords = (lat, lon)
                print(f"🔍 Scouting near: {display_location} with precise coordinates ({lat:.4f}, {lon:.4f})")
            else:
                print(f"🔍 Scouting near: {display_location} (address-based search)")

        # Priority 2: Use coordinates if no address but coordinates exist
        elif lat is not None and lon is not None:
            coords = (lat, lon)
            search_location = f"Location at {lat:.4f}, {lon:.4f}"
            display_location = f"Your location ({lat:.4f}, {lon:.4f})"
            params.location = display_location
            print(f"🔍 Scouting near: {display_location}")

        # Priority 3: Default fallback (should rarely happen)
        else:
            search_location = "Boston, MA"
            display_location = "Boston, MA (default)"
            coords = None
            params.location = search_location
            print(f"🔍 Scouting near: {display_location} (using fallback)")

        print(f"   📍 Final search params: location='{search_location}', coords={coords}")

        # Get user's past preferences for personalization (ENHANCED RAG)
        print(f"   👤 Looking up user profile for: '{user_id}'")
        user_profile = self.rag_system.get_user_preferences(user_id)
        if user_profile['has_history']:
            print(f"   ✅ Found user history: {user_profile['total_adventures']} past adventures")
            print(f"   📊 User preferences: {', '.join(user_profile['common_preferences'][:3])}")
            print(f"   ⭐ Average rating: {user_profile['average_rating']:.1f}/5")
        else:
            print(f"   📝 No history found for user {user_id}")

        # Get relevant Boston insider tips (ENHANCED RAG)
        query_text = f"{params.mood} {' '.join(params.preferences)} adventure in Boston"
        insider_tips = self.rag_system.get_relevant_tips(query_text, params.preferences, n_results=3)

        # Get weather
        weather = self.weather_api.get_current_weather(params.location, lat, lon)
        is_outdoor_ok = self.weather_api.is_outdoor_friendly(weather)

        # CONDITIONAL SUNRISE/SUNSET API INTEGRATION
        sunrise_info = None
        photography_tips = None

        if self.sunrise_sunset_api and coords:
            should_use_sunrise = self.sunrise_sunset_api.should_use_for_request(user_input, params.preferences)

            if should_use_sunrise:
                print(f"🌅 SUNRISE API ACTIVATED - User requested timing/photography features")
                try:
                    sunrise_info = self.sunrise_sunset_api.is_golden_hour(coords[0], coords[1])
                    photography_tips = self.sunrise_sunset_api.get_photography_recommendations(
                        coords[0], coords[1],
                        activity_type=' '.join(params.preferences)
                    )
                    print(f"   ✅ Golden hour status: {sunrise_info.get('is_golden_hour', False)}")
                    print(f"   📸 Photography tips: {len(photography_tips.get('recommendations', []))} recommendations")
                except Exception as e:
                    print(f"   ⚠️ Sunrise API error: {e}")
                    sunrise_info = None
                    photography_tips = None
            else:
                print(f"⏭️ Sunrise API skipped - No timing keywords detected")

        # Pass time_available to nearby_search for smart distance filtering
        pois = self.maps_api.nearby_search(
            location=search_location,
            preferences=params.preferences,
            exact_coords=coords,
            radius=2000,
            mood=params.mood,
            time_available=params.time_available
        )

        print(f"🖼️ Fetching images for {len(pois)} POIs...")

        # Convert to POIs with images
        enhanced_pois = []
        for poi in pois:
            try:
                images = self.photos_api.get_poi_images(poi.id, max_photos=2)
                print(f"   {poi.name}: {len(images)} images, {len(poi.reviews)} reviews")
            except:
                images = []

            enhanced_poi = EnhancedPOI(
                id=poi.id, name=poi.name, type=poi.type, description=poi.description,
                lat=poi.lat, lon=poi.lon, rating=poi.rating, price_level=poi.price_level,
                current_wait=poi.current_wait, tags=poi.tags, match_score=poi.match_score,
                score_reason=poi.score_reason, distance_from_user=poi.distance_from_user,
                images=images, reviews=poi.reviews
            )
            enhanced_pois.append(enhanced_poi)

        # Score POIs with ENHANCED RAG and sunrise context
        enhanced_pois = self._score_pois_with_enhanced_rag(
            enhanced_pois, params, weather, insider_tips, user_profile, sunrise_info, photography_tips
        )

        # ENHANCED CONTEXT with sunrise info
        context = {
            "weather": weather,
            "outdoor_friendly": is_outdoor_ok,
            "location_used": params.location,
            "coordinates_used": coords is not None,
            "insider_tips": insider_tips,
            "user_profile": user_profile,
            "sunrise_info": sunrise_info,  # NEW
            "photography_tips": photography_tips,  # NEW
            "sunrise_api_used": sunrise_info is not None  # NEW
        }

        return enhanced_pois[:8], context

    def _score_pois_with_enhanced_rag(self, pois: List[EnhancedPOI], params: AdventureParams,
                            weather: Dict, insider_tips: List[Dict], user_profile: Dict,
                            sunrise_info: Dict = None, photography_tips: Dict = None) -> List[EnhancedPOI]:
        """ENHANCED scoring with RAG, user history, and optional sunrise/sunset context"""

        # Format insider tips for prompt
        tips_text = "\n".join([f"- {tip['tip']}" for tip in insider_tips]) if insider_tips else "No specific insider tips available."

        # Enhanced user profile formatting
        profile_text = ""
        if user_profile['has_history']:
            profile_text = f"""
USER HISTORY (Enhanced Profile):
- Past preferences: {', '.join(user_profile['common_preferences'])}
- Typical budget: ${user_profile['average_budget']:.2f}
- Average time preference: {user_profile.get('average_time', 120)} minutes
- Favorite mood: {user_profile['favorite_mood']}
- Past adventure rating: {user_profile['average_rating']:.1f}/5
- Total adventures: {user_profile.get('total_adventures', 0)}
- Highly rated themes: {', '.join(user_profile.get('highly_rated_themes', [])[:2])}
Consider this comprehensive history but prioritize current request."""

        # NEW: Sunrise/sunset context
        timing_context = ""
        if sunrise_info:
            timing_context = f"""
TIMING & PHOTOGRAPHY CONTEXT:
- Golden hour status: {'Currently golden hour!' if sunrise_info.get('is_golden_hour') else 'Not golden hour'}
- Current time: {sunrise_info.get('current_time', 'Unknown')}
- Recommendation: {sunrise_info.get('recommendation', 'Great lighting available')}"""

            if photography_tips and photography_tips.get('recommendations'):
                timing_context += f"\n- Photography tips: {'; '.join(photography_tips['recommendations'][:2])}"

        # Enhanced scoring prompt with timing context
        scoring_prompt = f"""Score these locations for the user's request with enhanced context:

USER REQUEST:
- Mood: {params.mood}
- Time: {params.time_available} minutes ({'SHORT TRIP - prioritize nearby locations' if params.time_available <= 60 else 'LONGER TRIP - distance less critical'})
- Budget: ${params.budget}
- Preferences: {', '.join(params.preferences)}
- Location: {params.location}

WEATHER: {weather['weather'][0]['description']}, {weather['main']['temp']}°F

BOSTON INSIDER TIPS (from enhanced RAG database):
{tips_text}

{profile_text}

{timing_context}

LOCATIONS TO SCORE:
{json.dumps([{
    "name": p.name,
    "type": p.type,
    "rating": p.rating,
    "price": p.price_level,
    "description": p.description,
    "distance_km": round(p.distance_from_user, 1) if p.distance_from_user else "unknown"
} for p in pois], indent=2)}

Rate each location 0-100 based on:
- Match with user preferences (25 points)
- Alignment with insider tips (20 points)
- User history compatibility (15 points) - NEW enhanced factor
- Suitability for mood and time (15 points)
- Distance appropriateness (15 points) - CRITICAL for trips under 60 minutes
- Weather appropriateness (5 points)
- Budget fit (5 points)
{'- Golden hour/timing suitability (5 bonus points)' if sunrise_info else ''}

ENHANCED SCORING RULES:
- Prioritize locations matching user's highly rated themes from history
- For trips ≤30 minutes: Locations >2km away lose 20+ points
- For trips ≤60 minutes: Locations >3km away lose 15+ points
- If golden hour active: Outdoor locations get bonus points
- Match user's typical time/budget patterns when possible

Return JSON: [{{"name": "exact name", "score": 85, "reason": "detailed scoring rationale with history/timing context"}}]
Return ONLY the JSON."""

        try:
            response = router.route(TaskType.REASONING, scoring_prompt, temperature=0.4)

            if "```json" in response:
                response = response.split("```json")[1].split("```")[0]
            elif "```" in response:
                response = response.split("```")[1].split("```")[0]

            scores = json.loads(response.strip())

            # Apply scores
            for poi in pois:
                for score_data in scores:
                    if poi.name.lower() in score_data["name"].lower() or score_data["name"].lower() in poi.name.lower():
                        poi.match_score = score_data["score"]
                        poi.score_reason = score_data["reason"]
                        break

                # Enhanced default scoring with user history consideration
                if poi.match_score == 0:
                    base_score = 70

                    # Apply distance penalty
                    if poi.distance_from_user and params.time_available:
                        if params.time_available <= 30 and poi.distance_from_user > 2:
                            base_score -= 20
                        elif params.time_available <= 60 and poi.distance_from_user > 3:
                            base_score -= 15

                    # NEW: User history bonus
                    if user_profile['has_history']:
                        # Bonus for matching user's preferred POI types
                        if poi.type.lower() in [pref.lower() for pref in user_profile['common_preferences']]:
                            base_score += 10
                        # Bonus for matching highly rated themes
                        highly_rated = user_profile.get('highly_rated_themes', [])
                        if highly_rated and poi.type.lower() in [theme.lower() for theme in highly_rated]:
                            base_score += 5

                    # NEW: Golden hour bonus
                    if sunrise_info and sunrise_info.get('is_golden_hour') and poi.type in ['park', 'outdoor', 'waterfront']:
                        base_score += 5

                    poi.match_score = max(base_score, 30)

            pois.sort(key=lambda x: x.match_score, reverse=True)
            print(f"✅ Scored {len(pois)} POIs with enhanced RAG and timing awareness")

            return pois

        except Exception as e:
            print(f"⚠️ Enhanced scoring error: {e}")
            # Enhanced default scores with user history consideration
            for i, poi in enumerate(pois):
                base_score = 85 - (i * 5)

                # Distance penalty
                if poi.distance_from_user and params.time_available:
                    if params.time_available <= 30 and poi.distance_from_user > 2:
                        base_score -= 20
                    elif params.time_available <= 60 and poi.distance_from_user > 3:
                        base_score -= 15

                # User history bonus
                if user_profile['has_history'] and poi.type.lower() in [pref.lower() for pref in user_profile['common_preferences']]:
                    base_score += 10

                # Golden hour bonus
                if sunrise_info and sunrise_info.get('is_golden_hour') and poi.type in ['park', 'outdoor', 'waterfront']:
                    base_score += 5

                poi.match_score = max(base_score, 30)
            return pois

print("✅ Enhanced Scout Agent with RAG and conditional Sunrise/Sunset API")

✅ Enhanced Scout Agent with RAG and conditional Sunrise/Sunset API


In [ ]:
# Cell 12

# Optimizer Agent
class OptimizerAgent:
    """Create diverse adventure itineraries with dynamic themes"""

    def __init__(self):
        pass

    def optimize(self, params: AdventureParams, pois: List[EnhancedPOI], context: Dict) -> List[AdventureOption]:
        """Generate diverse itineraries that respect time constraints"""

        if len(pois) < 2:
            return self._create_simple_adventures(params, pois, context)

        # Strict time allocation based on available time
        if params.time_available <= 15:
            max_stops = 1
            activity_time = params.time_available - 5  # 5 min buffer
        elif params.time_available <= 30:
            max_stops = 2
            activity_time = params.time_available - 10  # 10 min travel/buffer
        elif params.time_available <= 60:
            max_stops = 2
            activity_time = params.time_available - 15  # 15 min travel/buffer
        else:
            max_stops = 3
            activity_time = params.time_available - 20  # 20 min travel/buffer

        print(f"🎯 TIME CONSTRAINTS: {params.time_available}min total → {max_stops} stops, {activity_time}min activity time")

        # Generate themes
        themes = self._generate_dynamic_themes(params, pois, {})

        optimization_prompt = f"""Create 3 DIFFERENT adventure itineraries with STRICT time limits:

    USER: {params.mood} mood, {params.time_available} min TOTAL, ${params.budget}
    PREFERENCES: {', '.join(params.preferences)}

    CRITICAL TIME CONSTRAINTS:
    - Maximum {max_stops} stops total
    - Total activity time: {activity_time} minutes
    - Include 5-10 minutes travel time between stops
    - NEVER exceed {params.time_available} minutes total

    AVAILABLE LOCATIONS (all within walking distance):
    {json.dumps([{
        "name": p.name,
        "type": p.type,
        "rating": p.rating,
        "distance_km": round(p.distance_from_user, 1) if p.distance_from_user else "nearby"
    } for p in pois[:8]], indent=2)}

    CREATE 3 ADVENTURES:
    1. **{themes[0]}** - Focus on closest location
    2. **{themes[1]}** - Different location/approach
    3. **{themes[2]}** - Alternative experience

    FOR EACH ADVENTURE, return JSON:
    {{
      "title": "descriptive title",
      "tagline": "one-line description",
      "stops": [
        {{
          "poi": "exact name from locations",
          "duration": {activity_time // max_stops},
          "activity": "specific activity description",
          "cost": 0.0,
          "why": "why this timing works"
        }}
      ],
      "total_duration": {params.time_available},
      "total_cost": 0.0,
      "theme": "{themes[0]}/{themes[1]}/{themes[2]}"
    }}

    Return JSON array with 3 adventures. RESPECT THE TIME LIMIT!"""
        try:
            response = router.route(TaskType.OPTIMIZATION, optimization_prompt,
                                   temperature=0.8, max_tokens=2500)

            if "```json" in response:
                response = response.split("```json")[1].split("```")[0]
            elif "```" in response:
                response = response.split("```")[1].split("```")[0]

            adventures_data = json.loads(response.strip())

            if not isinstance(adventures_data, list):
                adventures_data = [adventures_data]

            # Convert to AdventureOption objects
            adventures = []
            used_pois = set()

            for i, adv in enumerate(adventures_data[:3]):
                stops = adv.get("stops", [])

                # Check for POI overlap
                current_poi_names = {stop.get("poi", "") for stop in stops}
                if used_pois & current_poi_names and len(adventures) > 0:
                    print(f"⚠️ Adventure {i+1} overlaps POIs, skipping")
                    continue

                used_pois.update(current_poi_names)

                # Build steps
                steps = []
                cumulative_time = 0

                for stop in stops:
                    cumulative_time += 5  # Travel time
                    boston_tz = pytz.timezone('America/New_York')
                    arrival_time = datetime.now(boston_tz) + timedelta(minutes=cumulative_time)

                    steps.append({
                        "time": arrival_time.strftime('%I:%M %p'),
                        "activity": f"{stop['poi']}: {stop.get('activity', 'visit')}",
                        "details": f"Duration: {stop.get('duration', 30)} min • {stop.get('why', 'Great choice')}"
                    })

                    cumulative_time += stop.get('duration', 30)

                if not steps:  # Fallback
                    poi = pois[i % len(pois)]
                    steps = [{
                        "time": "Now",
                        "activity": f"{poi.name}: Explore",
                        "details": f"Duration: 30 min • {poi.type}"
                    }]

                # Get theme properly with validation
                claimed_theme = adv.get("theme", themes[i])

                # Validate theme matches one of our generated themes
                if claimed_theme not in themes:
                    print(f"⚠️ Theme mismatch! Got '{claimed_theme}', expected one of {themes}")
                    claimed_theme = themes[i]  # Force correct theme

                print(f"✅ Adventure {i+1}: '{adv.get('title')}' → Theme: {claimed_theme}")

                adventure = AdventureOption(
                    title=adv.get("title", f"Adventure {i+1}"),
                    tagline=adv.get("tagline", "Discover something new"),
                    description=adv.get("reasoning", "A unique experience"),
                    match_score=92 - (i * 4),
                    duration=adv.get("total_duration", cumulative_time),
                    cost=adv.get("total_cost", params.budget * 0.7),
                    distance=1.5 + i * 0.5,
                    steps=steps,
                    reasoning=claimed_theme,  # ← Use validated theme here
                    weather_dependent=not context['outdoor_friendly']
                )

                adventures.append(adventure)

            # Ensure we have 3 different adventures
            if len(adventures) < 3:
                print(f"⚠️ Only got {len(adventures)} adventures, creating fallbacks...")
                fallback_adventures = self._create_simple_adventures(params, pois, context)
                adventures.extend(fallback_adventures[:3-len(adventures)])

            return adventures

        except Exception as e:
            print(f"⚠️ Optimization error: {e}")
            import traceback
            traceback.print_exc()
            return self._create_simple_adventures(params, pois, context)

    def _generate_dynamic_themes(self, params: AdventureParams, pois: List[POI],
                                 type_counts: Dict) -> List[str]:
        """Generate appropriate themes based on user preferences and available POIs"""

        preferences = [p.lower() for p in params.preferences]

        # Shopping-focused themes
        if any(word in ' '.join(preferences) for word in ['shop', 'retail', 'boutique', 'mall', 'store']):
            return [
                "Luxury Boutique Tour",
                "Smart Shopper's Route",
                "Mall & Market Explorer"
            ]

        # Tourist/Sightseeing themes
        if any(word in ' '.join(preferences) for word in ['tourist', 'sightseeing', 'landmarks', 'iconic', 'memorable', 'visit', 'see boston', 'explore boston']):
            return [
                "Iconic Boston Experience",
                "Hidden Gems Tour",
                "Essential Boston Highlights"
            ]

        # Coffee-focused themes
        elif any(word in ' '.join(preferences) for word in ['coffee', 'cafe']):
            return [
                "Coffee Connoisseur Path",
                "Cafe & Culture Mix",
                "Budget Brew Tour"
            ]

        # Art/Culture themes
        elif any(word in ' '.join(preferences) for word in ['art', 'museum', 'gallery', 'culture']):
            return [
                "Cultural Deep Dive",
                "Art & Refreshments",
                "Quick Gallery Hop"
            ]

        # Nature/Outdoor themes
        elif any(word in ' '.join(preferences) for word in ['park', 'walk', 'nature', 'outdoor', 'trail']):
            return [
                "Nature Immersion",
                "Urban Green Escape",
                "Quick Outdoor Reset"
            ]

        # Food/Dining themes
        elif any(word in ' '.join(preferences) for word in ['food', 'restaurant', 'dining', 'eat']):
            return [
                "Culinary Adventure",
                "Taste of the City",
                "Budget Foodie Tour"
            ]

        # Nightlife themes
        elif any(word in ' '.join(preferences) for word in ['bar', 'pub', 'nightlife', 'drinks', 'music']):
            return [
                "Evening Entertainment",
                "Music & Drinks Circuit",
                "Casual Night Out"
            ]

        # Generic fallback
        else:
            return [
                "Premium Experience",
                "Balanced Explorer",
                "Budget-Friendly Route"
            ]

    def _get_theme_description(self, theme: str, params: AdventureParams) -> str:
        """Get description for each theme type"""

        theme_descriptions = {
            # Shopping themes
            "Luxury Boutique Tour": "High-end boutiques and designer shopping",
            "Smart Shopper's Route": "Best deals and value shopping spots",
            "Mall & Market Explorer": "Mix of mall shopping and unique markets",

            # Tourist/Sightseeing themes
            "Iconic Boston Experience": "Must-see landmarks and famous attractions",
            "Hidden Gems Tour": "Off-the-beaten-path local favorites",
            "Essential Boston Highlights": "Classic Boston in limited time",

            # Coffee themes
            "Coffee Connoisseur Path": "Premium coffee experiences",
            "Cafe & Culture Mix": "Coffee paired with cultural activities",
            "Budget Brew Tour": "Great coffee at affordable prices",

            # Art/Culture themes
            "Cultural Deep Dive": "Extensive museum and gallery exploration",
            "Art & Refreshments": "Art viewing with dining breaks",
            "Quick Gallery Hop": "Efficient art tour for time-conscious visitors",

            # Nature themes
            "Nature Immersion": "Extended outdoor experiences",
            "Urban Green Escape": "Parks and green spaces in the city",
            "Quick Outdoor Reset": "Brief nature break",

            # Food themes
            "Culinary Adventure": "Full dining experience across venues",
            "Taste of the City": "Sample diverse local cuisine",
            "Budget Foodie Tour": "Delicious food on a budget",

            # Nightlife themes
            "Evening Entertainment": "Full night out experience",
            "Music & Drinks Circuit": "Live music and bar hopping",
            "Casual Night Out": "Relaxed evening activities",

            # Generic themes
            "Premium Experience": "High-quality venues and activities",
            "Balanced Explorer": "Mix of different activity types",
            "Budget-Friendly Route": "Best value options"
        }

        return theme_descriptions.get(theme, "Unique local experience")

    def _create_simple_adventures(self, params: AdventureParams, pois: List[EnhancedPOI],
                                 context: Dict) -> List[AdventureOption]:
        """Create simple diverse adventures as fallback"""

        adventures = []
        used_pois = set()

        # Generate appropriate themes
        type_counts = {}
        for poi in pois:
            type_counts[poi.type] = type_counts.get(poi.type, 0) + 1

        themes = self._generate_dynamic_themes(params, pois, type_counts)

        for i in range(3):
            # Pick different POIs for each adventure
            available_pois = [p for p in pois if p.name not in used_pois]
            if not available_pois:
                available_pois = pois
                used_pois.clear()

            if available_pois:
                poi = available_pois[0]
                used_pois.add(poi.name)

                boston_tz = pytz.timezone('America/New_York')
                steps = [{
                    "time": datetime.now(boston_tz).strftime('%I:%M %p'),
                    "activity": f"{poi.name}: Enjoy the experience",
                    "details": f"Duration: 45 min • {poi.type} • {poi.rating}⭐"
                }]

                cost = (poi.price_level + 1) * 8 if poi.price_level > 0 else 0

                adventure = AdventureOption(
                    title=f"{themes[i]} Adventure",
                    tagline=f"Discover {poi.type} excellence",
                    description=f"Visit {poi.name}, a popular {poi.type}. {poi.description}",
                    match_score=85 - (i * 5),
                    duration=45,
                    cost=min(cost, params.budget),
                    distance=1.0 + i * 0.3,
                    steps=steps,
                    reasoning=themes[i],  # ← Use dynamic theme
                    weather_dependent=False
                )

                adventures.append(adventure)

        return adventures

print("✅ Optimizer Agent with Dynamic Themes defined")

✅ Optimizer Agent with Dynamic Themes defined


In [ ]:
# Cell 13

# Curator Agent
class CuratorAgent:
    """Polish adventures with rich narratives"""

    def curate(self, adventure: AdventureOption, params: AdventureParams, pois: List[EnhancedPOI] = None) -> AdventureOption:
        """Adventure with compelling narrative"""

        # Extract reviews from POIs if available
        review_context = ""
        if pois:
            for step in adventure.steps:
                step_name = step.get('activity', '').split(':')[0].strip()
                matching_poi = next((p for p in pois if p.name.lower() in step_name.lower()), None)

                if matching_poi and matching_poi.reviews:
                    # Get best review (highest rating)
                    best_review = max(matching_poi.reviews, key=lambda r: r['rating'])
                    review_context += f"\n- {matching_poi.name}: \"{best_review['text'][:100]}...\" - {best_review['author']}"

                    # Store review in adventure
                    if not adventure.review_example:  # Only set once
                        adventure.review_theme = f"{best_review['rating']}⭐ experience"
                        adventure.review_example = best_review['text'][:200]
                        adventure.review_rating = best_review['rating']
                        adventure.review_author = best_review['author']


        curation_prompt = f"""Transform this adventure into a compelling narrative:

ADVENTURE: {adventure.title}
MOOD: {params.mood}
STEPS: {json.dumps(adventure.steps, indent=2)}

REAL USER REVIEWS (use these to add authenticity):
{review_context if review_context else "No reviews available"}

Create:
1. A vivid 2-3 sentence description that captures the mood and incorporates review insights
2. A catchy tagline (max 8 words)

Style: Like a friend's enthusiastic recommendation with social proof.

Return JSON:
{{
  "description": "vivid 2-3 sentences with review-inspired details",
  "tagline": "catchy tagline"
}}

Return ONLY the JSON."""

        try:
            response = router.route(TaskType.CREATIVE, curation_prompt, temperature=0.9)

            if "```json" in response:
                response = response.split("```json")[1].split("```")[0]
            elif "```" in response:
                response = response.split("```")[1].split("```")[0]

            curated = json.loads(response.strip())

            adventure.description = curated.get("description", adventure.description)
            adventure.tagline = curated.get("tagline", adventure.tagline)

        except Exception as e:
            print(f"⚠️ Curation error: {e}")

        return adventure

print("✅ Curator Agent defined")

✅ Curator Agent defined


In [ ]:
# Cell 14

# Map Generator
class MapGenerator:
    """Clean embed + directions URLs"""

    def __init__(self, api_key: str):
        self.api_key = api_key

    def create_adventure_map(self, adventure: AdventureOption, pois: List[EnhancedPOI], user_location=None):
        """Create hybrid map with user address as starting point"""

        try:
            # Find POIs for this adventure
            adventure_pois = []
            for step in adventure.steps:
                step_name = step.get('activity', '').split(':')[0].strip()
                matching_poi = next((p for p in pois if p.name.lower() in step_name.lower()), None)
                if matching_poi:
                    adventure_pois.append(matching_poi)

            if not adventure_pois:
                return "<div style='color: #666; text-align: center; padding: 20px;'>📍 No locations to map</div>"

            # Get primary destination for embed
            primary_poi = adventure_pois[0]

            # Get user's actual address from agent_state
            user_address = None
            user_coords = None

            # Try to get the user's address from global agent_state
            if 'agent_state' in globals():
                user_address = agent_state.get("address_input")
                user_lat = agent_state.get("lat")
                user_lon = agent_state.get("lon")
                if user_lat is not None and user_lon is not None:
                    user_coords = (user_lat, user_lon)

            print(f"🗺️ MAP DEBUG:")
            print(f"   User address from agent_state: '{user_address}'")
            print(f"   User coordinates from agent_state: {user_coords}")
            print(f"   Adventure POIs: {[poi.name for poi in adventure_pois]}")

            # Build route stops starting with user's actual location
            route_coordinates = []
            location_names = []

            # Add user's specific address/coordinates as starting point
            if user_address and user_coords:
                # Use the specific address for a more precise starting point
                route_coordinates.append(f"{user_coords[0]},{user_coords[1]}")
                location_names.append(user_address)  # Use the actual address
                print(f"   ✅ Starting route from: {user_address} ({user_coords[0]:.4f}, {user_coords[1]:.4f})")
            elif user_location and 'lat' in user_location and 'lon' in user_location:
                # Fallback to passed user_location
                route_coordinates.append(f"{user_location['lat']},{user_location['lon']}")
                location_names.append("Your Location")
                print(f"   ✅ Starting route from passed location: ({user_location['lat']:.4f}, {user_location['lon']:.4f})")
            else:
                print(f"   ⚠️ No user starting point available")

            # Add unique POI coordinates
            seen_coords = set()
            for poi in adventure_pois:
                coord_key = (round(poi.lat, 4), round(poi.lon, 4))
                if coord_key not in seen_coords:
                    route_coordinates.append(f"{poi.lat},{poi.lon}")
                    location_names.append(poi.name)
                    seen_coords.add(coord_key)

            print(f"   📍 Complete route: {len(route_coordinates)} stops")
            for i, name in enumerate(location_names):
                print(f"      {i+1}. {name}")

            # Create embed URL for primary destination
            embed_place_name = primary_poi.name.replace(' ', '+').replace('&', 'and')
            embed_url = f"https://www.google.com/maps/embed/v1/place?key={self.api_key}&q={embed_place_name}&zoom=15"

            # Build directions URL with user address as origin
            if len(route_coordinates) >= 2:
                # Multi-stop route starting from user's location
                origin = route_coordinates[0]  # User's address/coordinates
                destination = route_coordinates[-1]  # Last POI

                if len(route_coordinates) > 2:
                    # With waypoints (intermediate POIs)
                    waypoints = route_coordinates[1:-1]
                    waypoints_param = "&waypoints=" + "%7C".join(waypoints)
                    complete_route_url = f"https://www.google.com/maps/dir/?api=1&origin={origin}&destination={destination}{waypoints_param}&travelmode=walking"
                    button_text = f"🗺️ Start Navigation ({len(route_coordinates)} stops)"
                    route_info_text = f"{len(route_coordinates)} total stops"
                else:
                    # Just origin (user) and destination (POI)
                    complete_route_url = f"https://www.google.com/maps/dir/?api=1&origin={origin}&destination={destination}&travelmode=walking"
                    button_text = f"🗺️ Get Directions to {primary_poi.name}"
                    route_info_text = f"From your location to {primary_poi.name}"
            else:
                # Single destination - use directions from user location
                if user_coords:
                    origin = f"{user_coords[0]},{user_coords[1]}"
                else:
                    origin = f"{primary_poi.lat},{primary_poi.lon}"  # Fallback
                destination = f"{primary_poi.lat},{primary_poi.lon}"
                complete_route_url = f"https://www.google.com/maps/dir/?api=1&origin={origin}&destination={destination}&travelmode=walking"
                button_text = f"🗺️ Get Directions"
                route_info_text = "1 destination"

            # Show starting address in the UI
            starting_location_display = user_address if user_address else "Your Location"

            # Generate the map HTML with user address prominently displayed
            map_html = f"""
            <div style="border: 1px solid #ddd; border-radius: 10px; padding: 15px; background: white; margin-bottom: 15px;">

                <!-- Adventure Title -->
                <h3 style="margin: 0 0 15px 0; color: #333; font-size: 16px; text-align: center;">
                    🗺️ {adventure.title}
                </h3>

                <!-- Embedded Map -->
                <div style="border-radius: 8px; overflow: hidden; margin-bottom: 15px;">
                    <iframe src="{embed_url}"
                            width="100%"
                            height="280"
                            style="border:0;"
                            allowfullscreen
                            loading="lazy">
                    </iframe>
                </div>

                <!-- Route Info with User Address -->
                <div style="background: #f5f5f5; padding: 12px; border-radius: 6px; margin-bottom: 15px; font-size: 13px; color: #555;">
                    <div style="text-align: center; margin-bottom: 8px; color: black;">
                        📍 <strong style="color: black;">Destination:</strong> {primary_poi.name}
                    </div>
                    <div style="text-align: center; margin-bottom: 8px; color: black;">
                        🏠 <strong style="color: black;">Starting from:</strong> {starting_location_display}
                    </div>
                    <div style="text-align: center; margin-bottom: 8px; color: black;">
                        🛤️ <strong style="color: black;">Route:</strong> {route_info_text}
                    </div>
                    <div style="font-size: 11px; color: #666; text-align: center;">
                        Full route: {" → ".join(location_names[:3])}{"..." if len(location_names) > 3 else ""}
                    </div>
                </div>

                <!-- DIRECTIONS BUTTON -->
                <div style="text-align: center; margin-bottom: 10px;">
                    <a href="{complete_route_url}"
                      target="_blank"
                      style="display: inline-block; background: #4285f4; color: white; padding: 15px 25px; border-radius: 8px; text-decoration: none; font-weight: 600; font-size: 14px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
                        {button_text}
                    </a>
                </div>

                <!-- Helper Text -->
                <div style="text-align: center; font-size: 11px; color: #888;">
                    💡 Opens Google Maps with walking route from {starting_location_display}
                </div>

            </div>
            """

            print(f"✅ Map starting from: {starting_location_display}")
            print(f"🔗 Directions URL: {complete_route_url}")
            return map_html

        except Exception as e:
            print(f"❌ Map error: {e}")
            import traceback
            traceback.print_exc()
            return f"<div style='padding: 15px; color: red; border: 1px solid #ffcdd2; border-radius: 8px; background: #ffebee;'>❌ Map generation failed: {str(e)}</div>"

print("✅ Map Generator defined")

✅ Map Generator defined


In [ ]:
# Cell 15

# ReAct Agent Tools
from langchain.tools import tool

# Global variables for agents
maps_api = None
weather_api = None
photos_api = None
parser = None
scout = None
optimizer = None
curator = None
map_generator = None

# Store state across tool calls
agent_state = {
    "params": None,
    "pois": [],
    "adventures": [],
    "context": {},
    "maps_html": [],
    "user_id": "demo_user"
}

def initialize_agents():
    """Initialize all agents for tools to use"""
    global maps_api, weather_api, photos_api, parser, scout, optimizer, curator, map_generator

    maps_api = GoogleMapsAPI(GOOGLE_MAPS_KEY)
    weather_api = WeatherAPI()
    photos_api = GooglePlacesPhotosAPI(GOOGLE_MAPS_KEY)
    map_generator = MapGenerator(GOOGLE_MAPS_KEY)

    parser = ConversationalIntentParser()
    scout = ScoutAgent(maps_api, weather_api, photos_api, rag_system)
    optimizer = OptimizerAgent()
    curator = CuratorAgent()

    print("✅ Agents initialized for ReAct tools")

@tool
def parse_user_intent(user_input: str) -> str:
    """
    Parse user's adventure request into structured parameters like mood, time, budget, and preferences.
    Use this when you need to understand what the user wants from their natural language request.
    Input: Natural language request (e.g., 'Coffee shops near me, 2 hours, $30, cozy')
    Output: Structured information about the user's preferences
    """
    try:
        print(f"\n🔧 TOOL CALLED: parse_user_intent")
        params, missing = parser.parse_with_clarification(user_input)

        if params is None:
            params = AdventureParams(
                mood="cozy", time_available=120, budget=30.0, location="Boston, MA",
                energy_level="medium", preferences=["coffee"], constraints=[],
                weather_preference="flexible"
            )

        agent_state["params"] = params

        result = f"Successfully parsed the request. Here's what I found:\n"
        result += f"- Mood: {params.mood}\n"
        result += f"- Time Available: {params.time_available} minutes\n"
        result += f"- Budget: ${params.budget}\n"
        result += f"- Preferences: {', '.join(params.preferences)}\n"
        result += f"- Location: {params.location}\n"
        result += f"Now you know what the user wants. Next, you might want to find locations that match these preferences."

        return result

    except Exception as e:
        return f"Error parsing request: {str(e)}"

@tool
def scout_locations(location_override: str = "") -> str:
    """
    Search for real locations using Google Maps that match user preferences.
    Enhanced with RAG personalization and conditional sunrise/sunset timing.
    """
    try:
        print(f"\n🔧 ENHANCED TOOL CALLED: scout_locations")

        if agent_state["params"] is None:
            return "I cannot search for locations yet because I don't know what the user wants. Please call parse_user_intent first."

        params = agent_state["params"]
        user_input = agent_state.get("user_input", "")  # Get user input for sunrise API

        # Use location data from agent_state if available
        address_input = agent_state.get("address_input", None)
        lat = agent_state.get("lat", None)
        lon = agent_state.get("lon", None)

        print(f"🔧 ENHANCED SCOUT TOOL DEBUG:")
        print(f"   address_input from agent_state: '{address_input}'")
        print(f"   lat from agent_state: {lat}")
        print(f"   lon from agent_state: {lon}")
        print(f"   location_override: '{location_override}'")
        print(f"   user_input: '{user_input}'")

        # Use location override if provided, otherwise use agent_state data
        if location_override:
            address_input = location_override
            print(f"   Using location override: {location_override}")

        # ENHANCED: Call scout with user_input for sunrise API conditional logic
        enhanced_pois, context = scout.scout_with_images(
            params,
            user_id=agent_state["user_id"],
            address_input=address_input,
            lat=lat,
            lon=lon,
            user_input=user_input  # ✅ CRITICAL FIX: Pass user input for sunrise API decision
        )

        agent_state["pois"] = enhanced_pois
        agent_state["context"] = context

        if len(enhanced_pois) == 0:
            return "I couldn't find any locations matching the user's preferences. You might want to suggest trying different criteria."

        # ENHANCED: Build result with RAG and timing context
        result = f"Excellent! I found {len(enhanced_pois)} personalized locations:\n\n"

        # Show user personalization info
        user_profile = context.get('user_profile', {})
        if user_profile.get('has_history'):
            result += f"📊 **Personalized for {agent_state['user_id']}** (based on {user_profile.get('total_adventures', 0)} past adventures)\n"
            result += f"💚 Your usual preferences: {', '.join(user_profile.get('common_preferences', [])[:2])}\n"
            result += f"⭐ Your average rating: {user_profile.get('average_rating', 0):.1f}/5\n\n"

        # Show sunrise/timing info if activated
        sunrise_info = context.get('sunrise_info')
        if sunrise_info:
            result += f"🌅 **Golden Hour Status:** {sunrise_info.get('recommendation', 'Great timing!')}\n"
            if sunrise_info.get('is_golden_hour'):
                result += f"✨ Perfect lighting for outdoor activities right now!\n"
            result += f"🕐 Current time: {sunrise_info.get('current_time', 'Now')}\n\n"

        # Show locations with enhanced details
        for i, poi in enumerate(enhanced_pois[:5], 1):
            distance_text = f" ({poi.distance_from_user:.1f}km away)" if poi.distance_from_user else ""
            result += f"{i}. **{poi.name}**{distance_text}\n"
            result += f"   - Type: {poi.type} | Rating: {poi.rating}⭐\n"
            result += f"   - Match Score: {poi.match_score}/100\n"
            result += f"   - Why: {poi.score_reason}\n"

            # Show image info
            if poi.images:
                result += f"   - Photos: {len(poi.images)} available\n"

            # Show review preview
            if poi.reviews:
                best_review = max(poi.reviews, key=lambda r: r['rating'])
                review_preview = best_review['text'][:80] + "..." if len(best_review['text']) > 80 else best_review['text']
                result += f"   - Review: \"{review_preview}\" - {best_review['author']} ({best_review['rating']}⭐)\n"

            result += "\n"

        if len(enhanced_pois) > 5:
            result += f"... and {len(enhanced_pois) - 5} more carefully selected locations.\n\n"

        # Show insider tips if available
        insider_tips = context.get('insider_tips', [])
        if insider_tips:
            result += f"💡 **Boston Insider Tips:**\n"
            for tip in insider_tips[:2]:
                result += f"- {tip['tip'][:100]}{'...' if len(tip['tip']) > 100 else ''}\n"
            result += "\n"

        # Show photography tips if sunrise API was used
        photography_tips = context.get('photography_tips')
        if photography_tips and photography_tips.get('recommendations'):
            result += f"📸 **Photography Recommendations:**\n"
            for tip in photography_tips['recommendations'][:2]:
                result += f"- {tip}\n"
            result += "\n"

        result += f"These locations are intelligently filtered based on your preferences, past history, and current conditions. Next, you might want to create adventure itineraries."

        return result

    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"Error in enhanced location scouting: {str(e)}"

print("✅ FIXED Scout Tool with proper user_input passing")

@tool
def create_adventure_itineraries() -> str:
    """
    Create 3 diverse adventure itineraries from found locations.
    Use this after you have found locations (after scout_locations).
    Each adventure will have a different theme and use different places.
    Output: Summary of created adventures
    """
    try:
        print(f"\n🔧 TOOL CALLED: create_adventure_itineraries")

        if agent_state["params"] is None or len(agent_state["pois"]) == 0:
            return "I cannot create adventures yet because I don't have locations to work with. Please call scout_locations first."

        adventures = optimizer.optimize(
            agent_state["params"],
            agent_state["pois"],
            agent_state["context"]
        )

        agent_state["adventures"] = adventures

        if len(adventures) == 0:
            return "I couldn't create adventures from the found locations. This is unusual - there might be an issue."

        result = f"Excellent! I've created {len(adventures)} diverse adventure options:\n\n"
        for i, adv in enumerate(adventures, 1):
            result += f"Adventure {i}: {adv.title}\n"
            result += f"   - Tagline: {adv.tagline}\n"
            result += f"   - Duration: {adv.duration} minutes\n"
            result += f"   - Cost: ${adv.cost:.2f}\n"
            result += f"   - Match Score: {adv.match_score}%\n\n"

        result += "The adventures are created but might need better descriptions. You might want to polish them next."

        return result

    except Exception as e:
        return f"Error creating adventures: {str(e)}"

@tool
def polish_adventure_descriptions() -> str:
    """
    Adventure descriptions with compelling, vivid narratives using real user reviews.
    """
    try:
        print(f"\n🔧 TOOL CALLED: polish_adventure_descriptions")

        if len(agent_state["adventures"]) == 0:
            return "I don't have any adventures to polish yet. Please call create_adventure_itineraries first."

        curated_adventures = []
        for adventure in agent_state["adventures"]:
            # Pass POIs so curator can access reviews
            curated = curator.curate(
                adventure,
                agent_state["params"],
                pois=agent_state.get("pois", [])
            )
            curated_adventures.append(curated)

        agent_state["adventures"] = curated_adventures

        result = f"Perfect! I've enhanced all {len(curated_adventures)} adventure descriptions with real user reviews.\n"
        result += "Each adventure now has vivid, compelling descriptions backed by authentic traveler experiences.\n\n"

        for i, adv in enumerate(curated_adventures, 1):
            result += f"Adventure {i}: {adv.title}\n"
            result += f'   - New Tagline: "{adv.tagline}"\n'
            if adv.review_example:
                result += f'   - Featured Review: {adv.review_rating}⭐ by {adv.review_author}\n'
            result += '\n'

        result += "The adventures are looking great! You might want to add maps for navigation next."

        return result

    except Exception as e:
        return f"Error polishing descriptions: {str(e)}"

@tool
def generate_adventure_maps() -> str:
    """
    Generate interactive Google Maps with turn-by-turn directions for each adventure.
    Now uses user's actual address as starting point.
    """
    try:
        print(f"\n🔧 TOOL CALLED: generate_adventure_maps")

        if len(agent_state["adventures"]) == 0:
            return "I don't have any adventures to create maps for. Please create adventures first."

        # Get user location data from agent_state
        user_address = agent_state.get("address_input")
        user_lat = agent_state.get("lat")
        user_lon = agent_state.get("lon")

        user_location_data = None
        if user_address and user_lat is not None and user_lon is not None:
            user_location_data = {
                "address": user_address,
                "lat": user_lat,
                "lon": user_lon
            }
            print(f"   🏠 Using user location: {user_address} ({user_lat:.4f}, {user_lon:.4f})")

        maps_html = []
        for adventure in agent_state["adventures"]:
            map_html = map_generator.create_adventure_map(
                adventure,
                agent_state["pois"],
                user_location_data
            )
            maps_html.append(map_html)

        agent_state["maps_html"] = maps_html

        result = f"Fantastic! I've generated {len(maps_html)} interactive maps with directions.\n"
        result += "Each map includes:\n"
        result += f"- Starting point: {user_address if user_address else 'Your location'}\n"
        result += "- Turn-by-turn walking directions from your address\n"
        result += "- Clickable buttons to open in Google Maps app\n\n"
        result += "The adventure planning is now complete!"

        return result

    except Exception as e:
        return f"Error generating maps: {str(e)}"

adventure_tools = [
    parse_user_intent,
    scout_locations,
    create_adventure_itineraries,
    polish_adventure_descriptions,
    generate_adventure_maps
]

print("✅ ReAct agent tools defined")

✅ FIXED Scout Tool with proper user_input passing
✅ ReAct agent tools defined


In [ ]:
# Cell 16 - Enhanced Main Orchestrator with Integrated RAG and Sunrise API

class ReActMiniQuestOrchestrator:
    """Main orchestrator using ReAct Agent - Enhanced with RAG and conditional Sunrise API"""

    def __init__(self):
        print("🚀 Initializing Enhanced ReAct MiniQuest (OpenAI + Claude + RAG + Sunrise)...")

        # Initialize enhanced agents with sunrise API
        self._initialize_enhanced_agents()

        self.agent = create_react_agent(
            model=ChatOpenAI(
                model="gpt-4o",
                temperature=0.7,
                api_key=OPENAI_API_KEY
            ),
            tools=adventure_tools
        )

        print("✅ Enhanced ReAct agent orchestrator initialized!")

    def _initialize_enhanced_agents(self):
        """Initialize all agents with enhanced RAG and sunrise API integration"""
        global maps_api, weather_api, photos_api, parser, scout, optimizer, curator, map_generator

        # Initialize APIs (same as before)
        maps_api = GoogleMapsAPI(GOOGLE_MAPS_KEY)
        weather_api = WeatherAPI()
        photos_api = GooglePlacesPhotosAPI(GOOGLE_MAPS_KEY)
        map_generator = MapGenerator(GOOGLE_MAPS_KEY)

        # Initialize agents with ENHANCED dependencies
        parser = ConversationalIntentParser()

        # ENHANCED Scout Agent with sunrise API
        scout = ScoutAgent(
            maps_api,
            weather_api,
            photos_api,
            rag_system,  # Enhanced RAG system
            sunrise_sunset_api  # NEW: Conditional sunrise API
        )

        optimizer = OptimizerAgent()
        curator = CuratorAgent()

        print("✅ Enhanced agents initialized with RAG and Sunrise API")

    def generate_adventures(self, user_input: str, user_id: str = "demo_user",
                           address_input: str = None, lat: float = None, lon: float = None):
        """Generate adventures using Enhanced ReAct Agent with RAG and Sunrise API"""

        # SET USER ID IN AGENT STATE
        agent_state["user_id"] = user_id

        # STORE USER INPUT for sunrise API conditional logic
        agent_state["user_input"] = user_input

        router.clear_traces()

        final_address = address_input
        final_lat = lat
        final_lon = lon

        # Use the address and coordinates as provided without extra geocoding
        if address_input and address_input.strip():
            final_address = address_input.strip()
            if lat is not None and lon is not None:
                # We have both address and coordinates - perfect!
                print(f"🏠 Using address: {final_address} with coordinates ({lat:.4f}, {lon:.4f})")
                final_lat = lat
                final_lon = lon
                location_desc = final_address
            else:
                # Only have address, no coordinates
                print(f"🏠 Using address only: {final_address}")
                location_desc = final_address
        elif lat is not None and lon is not None:
            # Only have coordinates
            final_lat = lat
            final_lon = lon
            final_address = f"Location: {lat:.4f}, {lon:.4f}"
            location_desc = final_address
            print(f"📍 Using coordinates only: ({final_lat:.4f}, {final_lon:.4f})")
        else:
            # No location info
            location_desc = "Boston, MA (default)"
            print(f"📍 Using default location: {location_desc}")

        # ENHANCED: Set location data in agent state for tools
        agent_state["address_input"] = final_address
        agent_state["lat"] = final_lat
        agent_state["lon"] = final_lon

        # ENHANCED: Check if user wants timing/photography features
        timing_requested = False
        if sunrise_sunset_api:
            timing_requested = sunrise_sunset_api.should_use_for_request(user_input, [])
            if timing_requested:
                print("🌅 User requested timing features - sunrise API will be activated")

        # Enhanced request message with timing context
        request_message = f"""A user wants to plan an adventure in Boston. Here's their request:

"{user_input}"

Location: {location_desc}
{f"Note: User requested timing/photography features - consider golden hour recommendations" if timing_requested else ""}

Help them plan a great adventure. You have tools available to understand their preferences, find locations, create itineraries, polish descriptions, and add maps. The system has enhanced RAG capabilities for personalization and optional sunrise/sunset timing features. Figure out the best approach and complete the task."""

        print("\n🤖 Running Enhanced ReAct Agent...")
        print("=" * 50)

        try:
            result = self.agent.invoke({
                "messages": [HumanMessage(content=request_message)]
            })

            messages = result.get("messages", [])

            print("\n🧠 Agent Thought Process:")
            for i, msg in enumerate(messages):
                if isinstance(msg, AIMessage):
                    content = msg.content if hasattr(msg, 'content') else str(msg)
                    if content and len(content.strip()) > 10:
                        print(f"  Step {i}: {content[:150]}...")

            print("\n" + "=" * 50)
            print("✅ Enhanced ReAct agent completed!")

            if len(agent_state["adventures"]) > 0:
                print(f"✅ Agent created {len(agent_state['adventures'])} adventures")

                # ENHANCED: Get context with sunrise info
                context = agent_state.get("context", {})
                sunrise_used = context.get("sunrise_api_used", False)

                enhanced_metadata = {
                    "params": agent_state["params"],
                    "context": context,
                    "pipeline_log": [
                        "Enhanced ReAct agent executed",
                        f"RAG system: {len(context.get('insider_tips', []))} insider tips",
                        f"User profile: {'Found' if context.get('user_profile', {}).get('has_history') else 'New user'}",
                        f"Sunrise API: {'Used' if sunrise_used else 'Not requested'}"
                    ],
                    "model_stats": router.get_stats(),
                    "enhanced_pois": agent_state.get("pois", []),
                    "total_images": sum(len(p.images) for p in agent_state.get("pois", [])),
                    "maps_generated": len(agent_state.get("maps_html", [])),
                    "sunrise_features_used": sunrise_used,  # NEW
                    "user_has_history": context.get('user_profile', {}).get('has_history', False),  # NEW
                    "insider_tips_count": len(context.get('insider_tips', []))  # NEW
                }

                return (
                    agent_state["adventures"],
                    enhanced_metadata,
                    agent_state.get("maps_html", [])
                )
            else:
                return self._create_enhanced_placeholder_response(user_input, location_desc, timing_requested)

        except Exception as e:
            print(f"❌ Enhanced ReAct agent error: {e}")
            import traceback
            traceback.print_exc()
            return [], {"error": str(e), "system": "Enhanced ReAct agent"}, []

    def _create_enhanced_placeholder_response(self, user_input, location, timing_requested):
        """Create enhanced placeholder response with timing context"""

        # Get sunrise info if timing was requested
        timing_note = ""
        if timing_requested and sunrise_sunset_api and agent_state.get("lat") and agent_state.get("lon"):
            try:
                golden_info = sunrise_sunset_api.is_golden_hour(agent_state["lat"], agent_state["lon"])
                timing_note = f" {golden_info.get('recommendation', '')}"
            except:
                timing_note = " Great lighting available throughout the day!"

        adventure = AdventureOption(
            title="Enhanced ReAct Adventure",
            tagline="AI-powered with RAG and timing intelligence",
            description=f"The enhanced ReAct agent analyzed your request: '{user_input}' and created this personalized adventure for {location}.{timing_note}",
            match_score=90,
            duration=120,
            cost=30.0,
            distance=2.0,
            steps=[
                {
                    "time": datetime.now(pytz.timezone('America/New_York')).strftime('%I:%M %p'),
                    "activity": "Enhanced Agent-planned adventure",
                    "details": f"Duration: 45 min • Personalized with RAG{' • Timing optimized' if timing_requested else ''}"
                }
            ],
            reasoning="Created through enhanced ReAct agent with RAG and timing awareness",
            weather_dependent=False
        )

        metadata = {
            "params": agent_state.get("params"),
            "context": agent_state.get("context", {}),
            "pipeline_log": ["Enhanced ReAct agent placeholder"],
            "model_stats": {"agent_calls": 1},
            "enhanced_pois": [],
            "total_images": 0,
            "maps_generated": 0,
            "sunrise_features_used": timing_requested,
            "user_has_history": False,
            "insider_tips_count": 0
        }

        return [adventure, adventure, adventure], metadata, []

print("✅ Enhanced ReAct MiniQuest Orchestrator with RAG and Sunrise API!")

# Initialize enhanced orchestrator
orchestrator = ReActMiniQuestOrchestrator()
print("✅ Enhanced ReAct MiniQuest Orchestrator instantiated!")

# Also create with clean_orchestrator name for backward compatibility
clean_orchestrator = orchestrator
print("✅ Enhanced backward compatibility alias created!")

✅ Enhanced ReAct MiniQuest Orchestrator with RAG and Sunrise API!
🚀 Initializing Enhanced ReAct MiniQuest (OpenAI + Claude + RAG + Sunrise)...
✅ Google Maps API initialized
✅ Enhanced agents initialized with RAG and Sunrise API
✅ Enhanced ReAct agent orchestrator initialized!
✅ Enhanced ReAct MiniQuest Orchestrator instantiated!
✅ Enhanced backward compatibility alias created!


/tmp/ipython-input-3970732998.py:12: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  self.agent = create_react_agent(


In [ ]:
# Cell 17.1 - Helper Functions

def process_detected_location(lat, lon):
    """Process auto-detected location from browser"""
    if lat is None or lon is None:
        return (
            "",
            '<div style="background: #e0f2fe; border: 1px solid #0284c7; color: #075985; padding: 12px; border-radius: 8px; margin: 10px 0;">📍 <strong>Location:</strong style="color: black;"> Ready for manual input</div>',
            None,
            None
        )

    try:
        import requests
        response = requests.get(
            f"https://api.bigdatacloud.net/data/reverse-geocode-client?latitude={lat}&longitude={lon}&localityLanguage=en",
            timeout=5
        )

        if response.status_code == 200:
            data = response.json()
            city = data.get('city', '')
            region = data.get('principalSubdivision', '')

            if city and region:
                address = f"{city}, {region}"
            else:
                address = f"Location: {lat:.4f}, {lon:.4f}"
        else:
            address = f"Location: {lat:.4f}, {lon:.4f}"

        status_html = f'<div style="background: #d1fae5; border: 1px solid #10b981; color: #065f46; padding: 12px; border-radius: 8px; margin: 10px 0;">✅ <strong>Auto-detected:</strong> {address}</div>'

        print(f"✅ Auto-detected location: {address}")
        return address, status_html, lat, lon

    except Exception as e:
        print(f"⚠️ Location detection error: {e}")
        address = f"Location: {lat:.4f}, {lon:.4f}"
        status_html = f'<div style="background: #fef3c7; border: 1px solid #f59e0b; color: #92400e; padding: 12px; border-radius: 8px; margin: 10px 0;">📍 <strong>Using coordinates:</strong> {address}</div>'
        return address, status_html, lat, lon


def check_user_history(username):
    """Check if user has history and display welcome message"""

    if not username or not username.strip():
        return '<div style="background: #e0f2fe; border: 1px solid #0284c7; color: #075985; padding: 10px; border-radius: 6px; margin: 10px 0;">💡 <strong>New user:</strong> Enter your name and we\'ll learn your preferences!</div>'

    user_id = username.strip()

    try:
        profile = rag_system.get_user_preferences(user_id)

        if profile['has_history']:
            prefs_text = ', '.join(profile['common_preferences'][:2]) if profile['common_preferences'] else 'Still learning...'
            return f'''<div style="background: #d1fae5; border: 1px solid #10b981; color: #065f46; padding: 10px; border-radius: 6px; margin: 10px 0;">
                ✅ <strong style="color: black;">Welcome back, {user_id}!</strong><br>
                📊 Adventures completed: {len(profile['adventures'])}<br>
                💚 You usually enjoy: {prefs_text}<br>
                ⭐ Your avg rating: {profile['average_rating']:.1f}/5
            </div>'''
        else:
            return f'<div style="background: #fef3c7; border: 1px solid #f59e0b; color: #92400e; padding: 10px; border-radius: 6px; margin: 10px 0;">👋 <strong style="color: black;">Welcome, {user_id}!</strong> This is your first adventure. We\'ll learn your preferences as you go!</div>'

    except Exception as e:
        print(f"⚠️ Error checking user history: {e}")
        return f'<div style="background: #fef3c7; border: 1px solid #f59e0b; color: #92400e; padding: 10px; border-radius: 6px; margin: 10px 0;">👋 <strong style="color: black;">Welcome, {user_id}!</strong> Ready to start your adventure!</div>'


def clear_chat():
    """Clear chat history and reset displays"""
    return [], "", None, []


print("✅ Helper functions defined")

✅ Helper functions defined


In [ ]:
# Cell 17.2 - Main Adventure Generation Function

def save_adventure_with_rating(user_name, adventure_idx, rating, adventures_data, params_data):
    """ENHANCED: Save user's completed adventure with rating using enhanced RAG system"""

    if not user_name or not user_name.strip():
        return "⚠️ Please enter your name to save preferences!"

    user_id = user_name.strip()

    try:
        # Get the selected adventure
        adventure = adventures_data[adventure_idx]
        params = params_data

        # ENHANCED: Store in RAG system with additional context
        adventure_id = rag_system.store_user_adventure(
            user_id=user_id,
            adventure=adventure,
            params=params,
            rating=rating,
            completion_status="completed",
            notes=""  # Could add user notes in the future
        )

        # Get updated user profile to show learning
        updated_profile = rag_system.get_user_preferences(user_id)

        # Enhanced feedback message
        response = f"""✅ **Adventure Saved & Learning Applied!**

**Adventure:** {adventure.title}
**Your Rating:** {'⭐' * int(rating)} ({rating}/5)
**Adventure ID:** {adventure_id[:8]}...

📊 **Your Updated Profile:**
- **Total Adventures:** {updated_profile.get('total_adventures', 1)}
- **Average Rating:** {updated_profile.get('average_rating', rating):.1f}/5
- **Top Preferences:** {', '.join(updated_profile.get('common_preferences', [])[:3])}
- **Favorite Mood:** {updated_profile.get('favorite_mood', params.mood)}

🎯 **What We Learned:**
- You enjoy {params.mood} adventures
- Your typical budget is ${updated_profile.get('average_budget', params.budget):.2f}
- You prefer {updated_profile.get('average_time', params.time_available)}-minute experiences"""

        # Add highly rated themes info
        highly_rated = updated_profile.get('highly_rated_themes', [])
        if highly_rated and rating >= 4:
            response += f"\n- You love {highly_rated[0]} themed experiences"

        response += f"\n\n🚀 **Next time you visit, we'll prioritize similar experiences!**"

        # Get personalized recommendations
        try:
            recommendations = rag_system.get_user_recommendations(user_id)
            if recommendations.get('recommendations'):
                response += f"\n\n💡 **Personal Recommendations:**"
                for rec in recommendations['recommendations'][:2]:
                    response += f"\n- {rec}"
        except:
            pass  # Skip if recommendations fail

        print(f"✅ Enhanced adventure save completed for {user_id}")
        return response

    except Exception as e:
        import traceback
        traceback.print_exc()
        return f"❌ Error saving adventure: {str(e)}\n\nPlease try again or contact support."

print("✅ Enhanced adventure save function with RAG integration")


def generate_clean_adventures(user_input, user_address, detected_lat_val, detected_lon_val,
                              user_name, chat_history):
    """Generate adventures based on user input with reviews"""

    if not user_input.strip():
        return chat_history, "", None, [], None, None

    if chat_history is None:
        chat_history = []

    chat_history.append({"role": "user", "content": user_input})

    try:
        # Determine user ID
        actual_user_id = user_name.strip() if user_name and user_name.strip() else "anonymous_user"
        print(f"\n👤 User: {actual_user_id}")

        # Determine location priority - manual address takes precedence
        final_address = None
        final_lat = None
        final_lon = None
        original_address = None

        if user_address and user_address.strip():
            # User entered manual address - USE THIS and geocode it
            original_address = user_address.strip()
            print(f"🏠 Using manual address: {original_address}")

            # Geocode the manual address to get coordinates
            try:
                location_data = location_detector.geocode_address(original_address)
                if location_data:
                    final_lat = location_data['lat']
                    final_lon = location_data['lon']
                    final_address = original_address  # Use original specific address
                    print(f"✅ Manual address geocoded: {original_address} → ({final_lat:.4f}, {final_lon:.4f})")
                else:
                    print("⚠️ Manual address geocoding failed, falling back to auto-detection")
                    final_address = None
            except Exception as e:
                print(f"⚠️ Geocoding error: {e}")
                final_address = None

        # Fallback to auto-detected location only if manual address failed
        if not final_address and detected_lat_val is not None and detected_lon_val is not None:
            final_lat = detected_lat_val
            final_lon = detected_lon_val
            final_address = f"Auto-detected location"
            print(f"📍 Using auto-detected location: ({final_lat:.4f}, {final_lon:.4f})")

        # CRITICAL FIX: Set the location data in agent_state BEFORE calling the agent
        if final_address or (final_lat is not None and final_lon is not None):
            print(f"🔧 FIXING: Setting agent location data...")
            print(f"   Address: {final_address}")
            print(f"   Coordinates: ({final_lat}, {final_lon})")

            # Set the global agent state with location information
            agent_state["address_input"] = final_address
            agent_state["lat"] = final_lat
            agent_state["lon"] = final_lon

            print(f"✅ Agent state updated with location data")

        # Generate adventures
        adventures, metadata, maps_html = clean_orchestrator.generate_adventures(
            user_input,
            user_id=actual_user_id,
            address_input=final_address,
            lat=final_lat,
            lon=final_lon
        )

        if not adventures:
            response = "❌ **Sorry!** I couldn't create adventures for that request. Please try with different parameters."
            chat_history.append({"role": "assistant", "content": response})
            return chat_history, "", None, [], None, None  # Return 6 values

        # Safe display location formatting with None checks
        if original_address:
            display_location = original_address
            location_source = "manual address"
        elif final_lat is not None and final_lon is not None:
            display_location = f"Location ({final_lat:.4f}, {final_lon:.4f})"
            location_source = "auto-detected"
        else:
            display_location = "Boston, MA (default)"
            location_source = "fallback"

        weather_desc = metadata['context']['weather']['weather'][0]['description']
        weather_temp = metadata['context']['weather']['main']['temp']

        # Show user profile info if available
        user_profile_text = ""
        if 'user_profile' in metadata['context'] and metadata['context']['user_profile']['has_history']:
            profile = metadata['context']['user_profile']
            user_profile_text = f"\n💡 *Based on your history, we know you enjoy: {', '.join(profile['common_preferences'][:2])}*\n"

        response = f"""✨ **Perfect!** I found **{len(adventures)} diverse adventures** for you:

**Your Request:** {metadata['params'].mood} • {metadata['params'].time_available} min • ${metadata['params'].budget}
**Location Used:** {display_location} *({location_source})*
**Weather:** {weather_desc.title()}, {weather_temp}°F
{user_profile_text}
---
"""

        gallery_images = []
        for idx, adventure in enumerate(adventures, 1):
            response += f"""
### 🌟 Option {idx}: {adventure.title} ({adventure.match_score}% match)
**{adventure.tagline}**

{adventure.description}

**Duration:** {adventure.duration} min • **Cost:** ${adventure.cost:.2f} • **Distance:** {adventure.distance:.1f} mi

**📍 Your Plan:**
"""

            for step in adventure.steps:
                response += f"• **{step['time']}** — {step['activity']}\n  *{step['details']}*\n"

                step_name = step.get('activity', '').split(':')[0].strip()
                matching_poi = next((p for p in metadata['enhanced_pois']
                                  if p.name.lower() in step_name.lower()), None)

                if matching_poi:
                    # Add images to gallery
                    if matching_poi.images:
                        for img in matching_poi.images[:2]:
                            gallery_images.append((img['url'], f"{matching_poi.name} - {matching_poi.type}"))

                    # Add best review for this location
                    if matching_poi.reviews:
                        best_review = max(matching_poi.reviews, key=lambda r: r['rating'])
                        review_text = best_review['text']

                        # Remove newlines and clean up the text
                        review_text = review_text.replace('\n', ' ').replace('\r', ' ')
                        review_text = ' '.join(review_text.split())

                        # Truncate long reviews
                        if len(review_text) > 120:
                            review_text = review_text[:120] + "..."
                        response += f"  💬 *\"{review_text}\" - {best_review['author']} ({best_review['rating']}⭐)*\n"

            response += f"\n*Theme: {adventure.reasoning}*\n"

            # Show featured review for the overall adventure
            if adventure.review_example:
                response += f"\n✨ **Featured Review:** {adventure.review_rating}⭐\n"
                review_preview = adventure.review_example

                review_preview = review_preview.replace('\n', ' ').replace('\r', ' ')
                review_preview = ' '.join(review_preview.split())

                if len(review_preview) > 200:
                    review_preview = review_preview[:200] + "..."
                response += f"*\"{review_preview}\" - {adventure.review_author}*\n"

            # Rating instruction
            if idx == 1:
                response += f"\n💡 **Tip:** After completing an adventure, rate it below so we can learn your preferences!\n"

            response += "---\n"

        # Add insider tips if available
        if 'insider_tips' in metadata['context'] and metadata['context']['insider_tips']:
            response += f"\n💡 **Boston Insider Tips:**\n"
            for tip in metadata['context']['insider_tips'][:2]:
                response += f"- {tip['tip']}\n"
            response += "\n"

        # NEW: Count total reviews fetched
        total_reviews = sum(len(p.reviews) for p in metadata.get('enhanced_pois', []))

        response += f"\n🎯 **System Stats:** {metadata['model_stats']['total_calls']} AI calls • {metadata.get('total_images', 0)} photos • {total_reviews} reviews • {metadata.get('maps_generated', 0)} maps\n\nCheck out your interactive maps on the right! 🗺️📸"

        chat_history.append({"role": "assistant", "content": response})

        # Build maps HTML
        final_maps_html = None
        if maps_html:
            maps_container = f"""
            <div style="background: #f8fafc; border-radius: 12px; padding: 20px; border: 1px solid #e2e8f0;">
                <h4 style="margin: 0 0 15px 0; color: #374151; display: flex; align-items: center; gap: 8px;">
                    <span style="background: #f59e0b; color: white; width: 28px; height: 28px; border-radius: 50%; display: flex; align-items: center; justify-content: center; font-size: 14px;">🗺️</span>
                    Your Adventure Maps ({len(maps_html)} available)
                </h4>
                <div style="max-height: 600px; overflow-y: auto;">
            """

            for i, map_html in enumerate(maps_html):
                adventure_title = adventures[i].title if i < len(adventures) else f"Adventure {i+1}"
                maps_container += f"""
                <div style="margin-bottom: 20px; border: 1px solid #e5e7eb; border-radius: 10px; overflow: hidden; background: white;">
                    <div style="background: linear-gradient(135deg, #f59e0b 0%, #d97706 100%); color: white; padding: 12px;">
                        <h3 style="margin: 0; font-size: 15px;">{adventure_title}</h3>
                    </div>
                    <div>
                        {map_html}
                    </div>
                </div>
                """

            final_maps_html = maps_container + "</div></div>"

        # Return exactly 6 values as expected by Gradio
        return chat_history, "", final_maps_html, gallery_images, adventures, metadata['params']

    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f"❌ DETAILED ERROR: {error_detail}")

        error_msg = f"❌ **Oops!** Something went wrong: {str(e)}\n\nPlease try again or check the console for details."
        chat_history.append({"role": "assistant", "content": error_msg})

        # Return exactly 6 values even in error case
        return chat_history, "", None, [], None, None

print("✅ Main generation function")

✅ Enhanced adventure save function with RAG integration
✅ Main generation function


In [ ]:
# Cell 17.3 - UI Layout with Event Handlers

def create_interface_layout():
    """Create the Gradio interface layout with event handlers"""

    with gr.Blocks(title="MiniQuest - AI Adventure Planning") as demo:

        # Header
        gr.HTML("""
        <div style="text-align: center; padding: 25px; background: linear-gradient(135deg, #f59e0b 0%, #d97706 100%); color: white; border-radius: 12px; margin-bottom: 25px; box-shadow: 0 4px 20px rgba(0,0,0,0.1);">
            <h1 style="margin: 0; font-size: 2.5rem; font-weight: 600;">MiniQuest</h1>
            <p style="margin: 15px 0 0 0; opacity: 0.9; font-size: 1.1rem;">AI-powered adventure planning • Personalized & Diverse</p>
        </div>
        """)

        with gr.Tabs():
            # Main Adventure Tab
            with gr.TabItem("🚀 Adventure Planner"):
                with gr.Row():
                    # Left Column - Inputs FIRST, then Chat
                    with gr.Column(scale=3):

                        # ===== USER IDENTITY SECTION (TOP) =====
                        with gr.Group():
                            gr.Markdown("### 👤 Your Profile")
                            user_name_input = gr.Textbox(
                                label="Your Name",
                                placeholder="Enter your name for personalization...",
                                value="",
                                lines=1,
                                info="We'll remember your preferences across sessions"
                            )
                            user_status = gr.HTML(
                                value='<div style="background: #e0f2fe; border: 1px solid #0284c7; color: #075985; padding: 10px; border-radius: 6px; margin: 10px 0;">💡 <strong>New user:</strong> Enter your name and we\'ll learn your preferences!</div>'
                            )

                        # ===== LOCATION SECTION (SECOND) =====
                        with gr.Group():
                            gr.Markdown("### 📍 Your Location")
                            user_address = gr.Textbox(
                                label="🏠 Your Address",
                                placeholder="Enter address or allow auto-detection...",
                                info="e.g., 'Downtown Boston' or '123 Main St, Boston, MA'",
                                lines=1
                            )
                            location_status = gr.HTML(
                                value='<div style="background: #fef3c7; border: 1px solid #f59e0b; color: #92400e; padding: 12px; border-radius: 8px; margin: 10px 0;">📍 <strong>Status:</strong> Ready for location input</div>'
                            )

                            # Hidden states
                            detected_lat = gr.State(value=None)
                            detected_lon = gr.State(value=None)

                        # ===== INPUT ROW (THIRD) =====
                        with gr.Row():
                            user_input = gr.Textbox(
                                placeholder="Try: 'Coffee shops near me, 2 hours, $30, cozy afternoon'",
                                label="What adventure are you looking for?",
                                show_label=True,
                                container=True,
                                scale=4,
                                lines=2
                            )
                            send_btn = gr.Button("Send", variant="primary", scale=1, size="lg")

                        # ===== CHAT INTERFACE (FOURTH - BELOW INPUTS) =====
                        chat_display = gr.Chatbot(
                            label="Adventure Chat",
                            height=350,
                            show_label=True,
                            container=True,
                            render_markdown=True,
                            type='messages',
                            allow_tags=False
                        )

                        clear_btn = gr.Button("🗑️ Clear Chat", variant="secondary", size="sm")

                        # ===== RATING SECTION (BOTTOM) =====
                        with gr.Group(visible=True):
                            gr.Markdown("### ⭐ Rate Your Adventure")
                            gr.Markdown("*After completing an adventure, rate it to help us learn your preferences!*")

                            with gr.Row():
                                adventure_selector = gr.Radio(
                                    choices=["Option 1", "Option 2", "Option 3"],
                                    label="Which adventure did you do?",
                                    value="Option 1"
                                )

                            with gr.Row():
                                rating_slider = gr.Slider(
                                    minimum=1,
                                    maximum=5,
                                    step=1,
                                    value=4,
                                    label="Your Rating (1-5 stars)",
                                    info="How much did you enjoy it?"
                                )

                            save_rating_btn = gr.Button("💾 Save Rating", variant="primary", size="sm")
                            rating_status = gr.Markdown("")

                            # Hidden states to store adventure data
                            stored_adventures = gr.State(value=None)
                            stored_params = gr.State(value=None)

                    # Right Column - Maps & Photos
                    with gr.Column(scale=2):
                        gr.Markdown("### 🗺️ Adventure Maps")
                        map_display = gr.HTML(
                            value="""<div style='text-align: center; padding: 50px 20px; color: #666; border: 2px dashed #ddd; border-radius: 12px; background: #f9fafb;'>
                                        <div style='font-size: 3rem; margin-bottom: 15px; opacity: 0.7;'>🗺️</div>
                                        <h3 style='margin: 0; color: #374151;'>Interactive Maps</h3>
                                        <p style='margin: 10px 0 0 0; font-size: 14px; color: #6b7280;'>
                                            Maps will appear here after adventure planning
                                        </p>
                                     </div>"""
                        )

                        gr.Markdown("### 📸 Location Photos")
                        image_gallery = gr.Gallery(
                            value=[],
                            label="Adventure Photos",
                            show_label=False,
                            columns=2,
                            rows=3,
                            object_fit="cover",
                            height=350
                        )

                # Examples Section
                gr.Markdown("### ☕ Try These Examples:")
                gr.Examples(
                    examples=[
                        ["Coffee shops near me, 2 hours, $30, cozy afternoon", "Boston, MA"],
                        ["Art galleries and cafes, 3 hours, $50, cultural exploration", "Cambridge, MA"],
                        ["Quick coffee break, 45 minutes, $15, energizing", "Downtown Boston"],
                        ["Bored on a Sunday, what should I do?", "Boston, MA"],
                        ["First date ideas that won't break the bank", "Back Bay, Boston"]
                    ],
                    inputs=[user_input, user_address],
                    examples_per_page=5
                )

            # About Tab
            with gr.TabItem("ℹ️ About"):
                gr.Markdown("""
                # ☕ MiniQuest - AI Adventure Planner

                ## 🎯 What is MiniQuest?

                MiniQuest is an **AI-powered adventure planning system** that creates personalized local experiences based on your preferences, time, budget, and location.

                ## ✨ Key Features

                ### 🤖 **Multi-Agent AI System**
                - **Intent Parser**: Understands natural language requests
                - **Scout Agent with RAG**: Finds locations using insider knowledge
                - **Optimizer Agent**: Creates diverse itineraries
                - **Curator Agent**: Polishes adventures with narratives

                ### 🧠 **Personalization & Learning**
                - **User History Tracking**: Remembers your past adventures
                - **Preference Learning**: Gets better at recommendations over time
                - **Boston Insider Tips**: 20+ curated local secrets in ChromaDB

                ### 🎯 **Guaranteed Diversity**
                - Each adventure uses **completely different locations**
                - Dynamic theme generation based on your interests
                - Smart algorithms ensure varied experiences

                ## 🛠️ **Technology Stack**

                - **AI Models**: OpenAI GPT-4/GPT-4-mini + Anthropic Claude Sonnet 4
                - **RAG System**: ChromaDB with semantic search
                - **Location Data**: Google Maps APIs (Places, Geocoding, Photos)
                - **Weather**: Weather.gov API
                - **Architecture**: LangGraph multi-agent pipeline

                ## 🚀 **How to Use**

                1. **Enter your name** for personalization
                2. **Set your location** (auto-detect or manual)
                3. **Type your request** naturally
                4. **Get 3 diverse adventures** with maps & photos
                5. **Rate your adventures** to improve recommendations

                ## 🏗️ **Built With Care**

                **Made with ☕ and 🤖 by Team 2**

                🙆‍♂️ Shreyas Sreenivas • 🙆‍♀️ Anagha Veena Sanjeev • 🙆‍♀️ Harshitha Chandrashekar
                """)

        # ========================================
        # EVENT HANDLERS - INSIDE gr.Blocks context
        # ========================================

        # Username change event - check user history
        user_name_input.change(
            fn=check_user_history,
            inputs=[user_name_input],
            outputs=[user_status]
        )

        # Send button click event
        send_btn.click(
            fn=generate_clean_adventures,
            inputs=[user_input, user_address, detected_lat, detected_lon, user_name_input, chat_display],
            outputs=[chat_display, user_input, map_display, image_gallery, stored_adventures, stored_params]
        )

        # Input field submit event
        user_input.submit(
            fn=generate_clean_adventures,
            inputs=[user_input, user_address, detected_lat, detected_lon, user_name_input, chat_display],
            outputs=[chat_display, user_input, map_display, image_gallery, stored_adventures, stored_params]
        )

        # Clear button event
        clear_btn.click(
            fn=clear_chat,
            outputs=[chat_display, user_address, map_display, image_gallery]
        )

        # Rating save event
        def handle_save_rating(user_name, adventure_choice, rating, adventures, params):
            if adventures is None:
                return "⚠️ Please generate adventures first!"

            # Map choice to index
            choice_map = {"Option 1": 0, "Option 2": 1, "Option 3": 2}
            idx = choice_map.get(adventure_choice, 0)

            result = save_adventure_with_rating(user_name, idx, int(rating), adventures, params)
            return result

        save_rating_btn.click(
            fn=handle_save_rating,
            inputs=[user_name_input, adventure_selector, rating_slider, stored_adventures, stored_params],
            outputs=[rating_status]
        )

        # Auto-detect location on load
        demo.load(
            fn=lambda: process_detected_location(None, None),
            inputs=[],
            outputs=[user_address, location_status, detected_lat, detected_lon],
            js="""
            function() {
                console.log('🔍 Starting location detection...');

                return new Promise((resolve) => {
                    if ('geolocation' in navigator) {
                        navigator.geolocation.getCurrentPosition(
                            function(position) {
                                const lat = position.coords.latitude;
                                const lon = position.coords.longitude;
                                console.log('✅ Location detected:', lat, lon);
                                resolve([lat, lon]);
                            },
                            function(error) {
                                console.log('❌ Location detection failed:', error.message);
                                resolve([null, null]);
                            },
                            { enableHighAccuracy: true, timeout: 15000, maximumAge: 300000 }
                        );
                    } else {
                        console.log('❌ Geolocation not supported');
                        resolve([null, null]);
                    }
                });
            }
            """
        )

    return demo


print("✅ UI layout")

✅ UI layout


In [ ]:
# Cell 17.4 - Launch Application

print("🔧 Creating MiniQuest interface...")

# Create the interface (event handlers already set up inside)
demo = create_interface_layout()

print("✅ Interface ready with personalization!")

# Launch the application
print("\n🚀 LAUNCHING MINIQUEST WITH RAG & USER PROFILES!")
print("="*50)

try:
    demo.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )

    print("\n🎉 MINIQUEST IS LIVE!")
    print("Features working:")
    print("✅ User personalization (enter your name)")
    print("✅ RAG with Boston insider tips")
    print("✅ User history tracking")
    print("✅ Diverse adventures (no duplicates)")
    print("✅ Interactive maps with directions")
    print("✅ Multi-LLM system (OpenAI + Claude)")
    print("✅ Dynamic theme generation")

except Exception as e:
    print(f"❌ Launch failed: {e}")
    import traceback
    traceback.print_exc()

print("="*50)

🔧 Creating MiniQuest interface...


/tmp/ipython-input-2297070221.py:67: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chat_display = gr.Chatbot(


✅ Interface ready with personalization!

🚀 LAUNCHING MINIQUEST WITH RAG & USER PROFILES!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://47e642285dcb647b2f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🔍 ENHANCED PROFILE LOOKUP:
   Original ID: 'W'
   Normalized ID: 'w'
   🔄 Falling back to individual adventure lookup...
   📝 No history found for user w

🔍 ENHANCED PROFILE LOOKUP:
   Original ID: 's'
   Normalized ID: 's'
   🔄 Falling back to individual adventure lookup...
   📝 No history found for user s

🔍 ENHANCED PROFILE LOOKUP:
   Original ID: 'sHREYAS s'
   Normalized ID: 'shreyas_s'
   🔄 Falling back to individual adventure lookup...
   📝 No history found for user shreyas_s

🔍 ENHANCED PROFILE LOOKUP:
   Original ID: 'sHREYAS sRE'
   Normalized ID: 'shreyas_sre'
   🔄 Falling back to individual adventure lookup...
   📝 No history found for user shreyas_sre

🔍 ENHANCED PROFILE LOOKUP:
   Original ID: 'S'
   Normalized ID: 's'
   🔄 Falling back to individual adventure lookup...
   📝 No history found for user s

🔍 ENHANCED PROFILE LOOKUP:
   Original ID: 'Shreyas'
   Normalized ID: 'shreyas'
   🔄 Falling back to individual adventure lookup...
   📝 No history found for user shreya